# <center>Bureau of Transportation Statistics Airline On-Time Performance Data Exploration</center>
## <center>by Marissa Mutare</center>

## Introduction

This project investigates the airline on-time performance of commercial domestic flights in the United States for the year 2006. The data used for this project can be found [here](https://www.google.com/url?q=http://stat-computing.org/dataexpo/2009/the-data.html&sa=D&ust=1554484977403000), where the complete set of data available ranges from 1987 to 2008.

> Introduce the dataset

>**Rubric Tip**: Your code should not generate any errors, and should use functions, loops where possible to reduce repetitive code. Prefer to use functions to reuse code statements.

> **Rubric Tip**: Document your approach and findings in markdown cells. Use comments and docstrings in code cells to document the code functionality.

>**Rubric Tip**: Markup cells should have headers and text that organize your thoughts, findings, and what you plan on investigating next.  

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from IPython.display import display

import matplotlib.patches as mpatches
from matplotlib import rcParams

%matplotlib inline

## Preliminary Wrangling
### Data Gathering
There are 4 CSV files that hold the data for this project. One CSV file holds the data about all commercial flights in the USA for the year 2006. Another CSV file holds data about the locations of USA airports. A third CSV file holds data about the carriers/airlines in the USA. The final CSV file holds data about individual aeroplanes.

All 4 files are conveniently already hosted on Kaggle, as part of the larger complete dataset from 1987 to 2008. Therefore, I will import the 4 files I need directly from the Kaggle data source into dataframes in the notebook. I'll then display the head of each dataframe to make sure the data was loaded correctly.

> Load in your dataset and describe its properties through the questions below. Try and motivate your exploration goals through this section.


In [2]:
# Loading in datasets
flights = pd.read_csv("../input/data-expo-2009-airline-on-time-data/2006.csv")
airports = pd.read_csv("../input/data-expo-2009-airline-on-time-data/airports.csv")
carriers = pd.read_csv("../input/data-expo-2009-airline-on-time-data/carriers.csv")
planes = pd.read_csv("../input/data-expo-2009-airline-on-time-data/plane-data.csv")

In [3]:
# Displaying head of each dataframe to verify that they all loaded successfully
print("\033[1mFlight Data:\033[0m")
display(flights.head())
print("\n\033[1mAirport Data:\033[0m")
display(airports.head())
print("\n\033[1mCarrier Data:\033[0m")
display(carriers.head())
print("\n\033[1mPlane Data:\033[0m")
display(planes.head())

Flight Data:


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,...,45,13,0,NaN,0,0,0,0,0,0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,...,27,19,0,NaN,0,0,0,0,0,0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,...,4,11,0,NaN,0,0,0,0,0,0
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,...,16,10,0,NaN,0,0,0,0,0,0
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,...,27,12,0,NaN,0,0,0,0,0,0



Airport Data:


,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944



Carrier Data:


,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.



Plane Data:


,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N050AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N051AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N052AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N054AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N055AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Assessment
In this subsection, I will assess the data for quality and tidiness, both visually and programmatically. I will document the full list of my observations towards the end of the subsection, below the displayed dataframes and assessment function calls.

Here, I will start by looking at the shape of the data.

In [4]:
print("\033[1mFlight Data Shape:\033[0m")
display(flights.shape)
print("\n\033[1mAirport Data Shape:\033[0m")
display(airports.shape)
print("\n\033[1mCarrier Data Shape:\033[0m")
display(carriers.shape)
print("\n\033[1mPlane Data Shape:\033[0m")
display(planes.shape)

Flight Data Shape:


(7141922, 29)


Airport Data Shape:


(3376, 7)


Carrier Data Shape:


(1491, 2)


Plane Data Shape:


(5029, 9)

The widest dataframe here is the`flights` dataframe with 29 columns. For the purposes of visual assessment, I would like to be able to view all columns without truncation. Therefore, I will now check the current setting for the maximum number of columns that will be displayed before Pandas collaspes the rest. If it is lower than 29, I will remove the limit for this notebook.

In [5]:
pd.get_option("display.max_columns")

20

In [6]:
pd.set_option("display.max_columns", None)

I can now go ahead with my inspection. For each of the 4 dataframes, I will now view the top few and bottom few rows, as part of visual assessment. I will also begin programmatic assessment, displaying the structure of each dataframe just below the dataframe itself for convenience of reference.

In [7]:
print("\033[1mFlight Data Summary: \033[0m")
display(flights)
flights.info(show_counts = True)
print("\n\033[1mAirport Data Summary: \033[0m")
display(airports)
airports.info()
print("\n\033[1mCarrier Data Summary: \033[0m")
display(carriers)
carriers.info()
print("\n\033[1mPlane Data Summary: \033[0m")
display(planes)
planes.info()

Flight Data Summary: 


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,N657AW,281.0,273.0,223.0,6.0,-2.0,ATL,PHX,1587,45,13,0,NaN,0,0,0,0,0,0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,N834AW,260.0,265.0,214.0,-5.0,0.0,ATL,PHX,1587,27,19,0,NaN,0,0,0,0,0,0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,N605AW,235.0,258.0,220.0,-23.0,0.0,ATL,PHX,1587,4,11,0,NaN,0,0,0,0,0,0
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,N312AW,152.0,158.0,126.0,-8.0,-2.0,AUS,PHX,872,16,10,0,NaN,0,0,0,0,0,0
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,N309AW,171.0,163.0,132.0,0.0,-8.0,AUS,PHX,872,27,12,0,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141917,2006,12,29,5,1246.0,1249,1452.0,1459,DL,1675,N905DE,126.0,130.0,108.0,-7.0,-3.0,ATL,EWR,745,6,12,0,NaN,0,0,0,0,0,0
7141918,2006,12,29,5,1225.0,1155,2033.0,1931,DL,1676,N651DL,308.0,276.0,261.0,62.0,30.0,SEA,ATL,2182,12,35,0,NaN,0,30,0,32,0,0
7141919,2006,12,29,5,2118.0,2115,2254.0,2241,DL,1676,N143DA,96.0,86.0,60.0,13.0,3.0,ATL,MCO,403,9,27,0,NaN,0,0,0,0,0,0
7141920,2006,12,29,5,2122.0,2127,2209.0,2223,DL,1677,N904DA,107.0,116.0,81.0,-14.0,-5.0,SLC,SJC,585,5,21,0,NaN,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141922 entries, 0 to 7141921
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               7141922 non-null  int64  
 1   Month              7141922 non-null  int64  
 2   DayofMonth         7141922 non-null  int64  
 3   DayOfWeek          7141922 non-null  int64  
 4   DepTime            7019988 non-null  float64
 5   CRSDepTime         7141922 non-null  int64  
 6   ArrTime            7003802 non-null  float64
 7   CRSArrTime         7141922 non-null  int64  
 8   UniqueCarrier      7141922 non-null  object 
 9   FlightNum          7141922 non-null  int64  
 10  TailNum            7141922 non-null  object 
 11  ActualElapsedTime  7003802 non-null  float64
 12  CRSElapsedTime     7141918 non-null  float64
 13  AirTime            7003802 non-null  float64
 14  ArrDelay           7003802 non-null  float64
 15  DepDelay           7019988 non-n

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   iata     3376 non-null   object 
 1   airport  3376 non-null   object 
 2   city     3364 non-null   object 
 3   state    3364 non-null   object 
 4   country  3376 non-null   object 
 5   lat      3376 non-null   float64
 6   long     3376 non-null   float64
dtypes: float64(2), object(5)
memory usage: 184.8+ KB

Carrier Data Summary: 


,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.
...,...,...
1486,ZW,Air Wisconsin Airlines Corp
1487,ZX,Air Georgian
1488,ZX (1),Airbc Ltd.
1489,ZY,Atlantic Gulf Airlines


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Code         1490 non-null   object
 1   Description  1491 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB

Plane Data Summary: 


,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N050AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N051AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N052AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N054AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N055AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5024,N997DL,Corporation,MCDONNELL DOUGLAS AIRCRAFT CO,03/11/1992,MD-88,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1992
5025,N998AT,Corporation,BOEING,01/23/2003,717-200,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2002
5026,N998DL,Corporation,MCDONNELL DOUGLAS CORPORATION,04/02/1992,MD-88,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1992
5027,N999CA,Foreign Corporation,CANADAIR,07/09/2008,CL-600-2B19,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1998


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5029 entries, 0 to 5028
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tailnum        5029 non-null   object
 1   type           4480 non-null   object
 2   manufacturer   4480 non-null   object
 3   issue_date     4480 non-null   object
 4   model          4480 non-null   object
 5   status         4480 non-null   object
 6   aircraft_type  4480 non-null   object
 7   engine_type    4480 non-null   object
 8   year           4480 non-null   object
dtypes: object(9)
memory usage: 353.7+ KB


For completeness and thoroughness, I also visually explored the CSV files directly in more depth using the Kaggle interface. I will now move forward with more programmatic assessment.

I will look at the descriptive statistics for each dataframe.

In [8]:
print("\033[1mFlight Data Descriptive Statistics:\033[0m")
display(flights.describe(include = "all"))
print("\n\033[1mAirport Data Descriptive Statistics:\033[0m")
display(airports.describe(include = "all"))
print("\n\033[1mCarrier Data Descriptive Statistics:\033[0m")
display(carriers.describe(include = "all"))
print("\n\033[1mPlane Data Descriptive Statistics:\033[0m")
display(planes.describe(include = "all"))

Flight Data Descriptive Statistics:


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7141922.0,7.141922e+06,7.141922e+06,7.141922e+06,7.019988e+06,7.141922e+06,7.003802e+06,7.141922e+06,7141922,7.141922e+06,7141922,7.003802e+06,7.141918e+06,7.003802e+06,7.003802e+06,7.019988e+06,7141922,7141922,7.141922e+06,7.141922e+06,7.141922e+06,7.141922e+06,121934,7.141922e+06,7.141922e+06,7.141922e+06,7.141922e+06,7.141922e+06,7.141922e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,5081,NaN,NaN,NaN,NaN,NaN,289,296,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WN,NaN,0,NaN,NaN,NaN,NaN,NaN,ATL,ATL,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1099321,NaN,72705,NaN,NaN,NaN,NaN,NaN,407971,404829,NaN,NaN,NaN,NaN,55655,NaN,NaN,NaN,NaN,NaN,NaN
mean,2006.0,6.562766e+00,1.573063e+01,3.949934e+00,1.340392e+03,1.332248e+03,1.486921e+03,1.495765e+03,NaN,2.186765e+03,NaN,1.260601e+02,1.271844e+02,1.028545e+02,8.682840e+00,1.009364e+01,NaN,NaN,7.279974e+02,7.061624e+00,1.574530e+01,1.707300e-02,NaN,2.266337e-03,3.396186e+00,6.796270e-01,3.585970e+00,3.099600e-02,4.515005e+00
std,0.0,3.424931e+00,8.786689e+00,1.991913e+00,4.765090e+02,4.625505e+02,5.008019e+02,4.786718e+02,NaN,1.989384e+03,NaN,7.131959e+01,7.031540e+01,7.644314e+01,3.657647e+01,3.350125e+01,NaN,NaN,5.745307e+02,3.112340e+01,1.127117e+01,1.295435e-01,NaN,4.755208e-02,1.880046e+01,8.492678e+00,1.562795e+01,1.277919e+00,1.986008e+01
min,2006.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,NaN,1.000000e+00,NaN,5.000000e+00,-9.700000e+01,-1.425000e+03,-5.920000e+02,-1.200000e+03,NaN,NaN,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2006.0,4.000000e+00,8.000000e+00,2.000000e+00,9.300000e+02,9.300000e+02,1.110000e+03,1.115000e+03,NaN,5.840000e+02,NaN,7.500000e+01,7.600000e+01,5.400000e+01,-9.000000e+00,-4.000000e+00,NaN,NaN,3.160000e+02,4.000000e+00,1.000000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2006.0,7.000000e+00,1.600000e+01,4.000000e+00,1.330000e+03,1.325000e+03,1.516000e+03,1.519000e+03,NaN,1.493000e+03,NaN,1.070000e+02,1.090000e+02,8.400000e+01,-1.000000e+00,0.000000e+00,NaN,NaN,5.740000e+02,5.000000e+00,1.300000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2006.0,1.000000e+01,2.300000e+01,6.000000e+00,1.732000e+03,1.720000e+03,1.911000e+03,1.906000e+03,NaN,3.442000e+03,NaN,1.560000e+02,1.570000e+02,1.320000e+02,1.300000e+01,1.000000e+01,NaN,NaN,9.560000e+02,8.000000e+00,1.800000e+01,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00



Airport Data Descriptive Statistics:


,iata,airport,city,state,country,lat,long
count,3376,3376,3364,3364,3376,3376.000000,3376.000000
unique,3376,3245,2675,56,5,NaN,NaN
top,00M,Municipal,Greenville,AK,USA,NaN,NaN
freq,1,5,11,263,3372,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,40.036524,-98.621205
std,NaN,NaN,NaN,NaN,NaN,8.329559,22.869458
min,NaN,NaN,NaN,NaN,NaN,7.367222,-176.646031
25%,NaN,NaN,NaN,NaN,NaN,34.688427,-108.761121
50%,NaN,NaN,NaN,NaN,NaN,39.434449,-93.599425
75%,NaN,NaN,NaN,NaN,NaN,43.372612,-84.137519



Carrier Data Descriptive Statistics:


,Code,Description
count,1490,1491
unique,1490,1491
top,02Q,Titan Airways
freq,1,1



Plane Data Descriptive Statistics:


,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
count,5029,4480,4480,4480,4480,4480,4480,4480,4480
unique,5029,5,39,2413,167,2,4,7,52
top,N050AA,Corporation,BOEING,12/22/1986,CL-600-2B19,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2001
freq,1,4440,2061,33,403,4479,4440,2883,389


I will check whether all airports in the `Origin` and `Dest` columns of the `flights` dataframe are present in the `airports` dataframe.

In [9]:
pd.Series(flights["Origin"].unique()).isin(airports["iata"]).value_counts()

True    289
dtype: int64

In [10]:
pd.Series(flights["Dest"].unique()).isin(airports["iata"]).value_counts()

True    296
dtype: int64

All values are True, and there are no False values. This means all airports in the `flights` dataframe are recorded in the `airports` dataframe.

I will now check whether all the carriers in the `UniqueCarrier` column of the `flights` dataframe are also in the `carriers` dataframe.

In [11]:
pd.Series(flights["UniqueCarrier"].unique()).isin(carriers["Code"]).value_counts()

True    20
dtype: int64

The values are True, and there are no False values. All carriers in the `flights` dataframe are recorded in the `carriers` dataframe.

I will now check whether all planes in the `TailNum` column of the `flights` dataframe are in the `planes` dataframe.

In [12]:
pd.Series(flights["TailNum"].unique()).isin(planes["tailnum"]).value_counts()

True     4245
False     836
dtype: int64

About 20% of the planes in the `flights` dataframe are not recorded in the `planes` dataframe. I will check how many of the flights used these planes that are not in the `planes` dataframe.

In [13]:
len(flights[~flights["TailNum"].isin(planes["tailnum"])])

1281391

These flights represent 18% of the flights in the dataframe. I will not drop them solely for the sake of one direction of inquiry and analysis of the plane variable, at the risk of lowering the quality of the analysis for the rest of the variables by reducing the amount of data used for them. I will instead revise how I conduct analysis of the individual planes and their contribution or relationship to flight delays.

The times in the `CRSDepTime` and `CRSArrTime` have the *int* data type. However, the times in the `DepTime` and `ArrTime` columns have the *float* data type. Since I will need to convert them to timestamps, I need to confirm whether all the times in the latter two columns are whole numbers, with only 0 after the decimal point.

In [14]:
(flights["DepTime"] % 1).sum()

0.0

In [15]:
(flights["ArrTime"] % 1).sum()

0.0

This is the case for both columns and is satisfactory.

For the columns `DepTime`, `CRSDepTime`, `ArrTime` and `CRSArrTime`, where times are stored as numerical values, I will now check whether the last two digits for each value, which represent the "minutes" portion of the time, are less than or equal to 59. I will do this by computing the values of the remainder after dividing by 100, and verifying that the highest of these values is 59.

In [16]:
(flights[["DepTime", "CRSDepTime", "ArrTime", "CRSArrTime"]] % 100).describe().loc[["min", "max"],]

,DepTime,CRSDepTime,ArrTime,CRSArrTime
min,0.0,0.0,0.0,0.0
max,59.0,59.0,59.0,59.0


The minimum value of the last two digits for each column is 0, and the maximum value is 59. This is satisfactory.

I will now check for duplicates in each dataframe.

In [17]:
flights.duplicated().sum()

7

In [18]:
airports.duplicated().sum()

0

In [19]:
carriers.duplicated().sum()

0

In [20]:
planes.duplicated().sum()

0

This concludes the visual and programmatic assessment section. I have documented the identified issues below.

#### Issues
`flights` dataframe
1. The `Year`, `Month` and `DayofMonth` columns are different parts of the same variable, the date.
2. The `DayOfWeek` column is not required.
3. The columns `DepTime`, `CRSDepTime`, `ArrTime`, and`CRSArrTime` have the *int* and *float* data types, which is incorrect.
4. Some of the `TailNum` values are 0 or 000000.
5. There are missing values in these columns: `DepTime`, `ArrTime`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`, `ArrDelay`, `DepDelay` and `CancellationCode`.
6. There are negative values in the `AirTime` column.
7. There are negative values in the `CRSElapsedTime` column.
8. There are values in the `DepTime` and `ArrTime` columns that are greater than 2359.
9. The scheduled departure times are not always on the same day as the actual departure times.
10. The origin airports are often not in the same timezone as the destination airports.
11. There are outliers in the `ActualElapsedTime`, `CRSElapsedTime`, `ArrDelay`, `DepDelay`, `TaxiIn`, `TaxiOut` and `Distance` columns.
12. There are duplicate entries in the dataframe.

`airports` dataframe

13. Some of the airports in the dataframe are not in the USA.
14. Some of the airports are duplicated in the dataframe.
15. There are missing values in these columns: `city` and `state`.

`carriers` dataframe

16. There is a missing value in 1 row in the `Code` column.

`planes` dataframe

17. The `issue_date` and`year` columns have the *string* data type, which is incorrect .
18. There are missing values in all columns except the first one.

### Data Cleaning

In this section, I will be addressing the issues that I discovered in the data and documented.

I start by making copies of the dataframes to work on.

In [29]:
# Copies of dataframes
flights_clean = flights.copy()
airports_clean = airports.copy()
carriers_clean = carriers.copy()
planes_clean = planes.copy()

#### Issue: 12. There are duplicate entries in the `flights` dataframe.

##### **Define**
- Identify rows that are duplicates.
- Drop identifed rows.
- Reset dataframe index.

##### **Code**

In [30]:
# Identify rows that are duplicates.
flights_clean[flights_clean.duplicated()]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
752508,2006,2,20,1,1204.0,1157,1244.0,1240,OO,6117,N297SW,40.0,43.0,28.0,4.0,7.0,PSP,LAX,110,3,9,0,NaN,0,0,0,0,0,0
888040,2006,2,19,7,821.0,730,948.0,850,FL,768,N956AT,87.0,80.0,50.0,58.0,51.0,PHL,BOS,280,15,22,0,NaN,0,0,0,7,0,51
1039373,2006,2,24,5,913.0,919,NaN,1054,AS,152,N746AS,NaN,95.0,NaN,NaN,-6.0,ANC,OME,539,0,17,0,NaN,1,0,0,0,0,0
1462456,2006,3,22,3,835.0,800,1101.0,1040,FL,762,N993AT,146.0,160.0,132.0,21.0,35.0,FLL,PHL,992,4,10,0,NaN,0,21,0,0,0,0
2653293,2006,5,26,5,2347.0,1951,59.0,2115,FL,467,N937AT,72.0,84.0,55.0,224.0,236.0,BWI,CLT,361,8,9,0,NaN,0,224,0,0,0,0
3243628,2006,6,10,6,1405.0,1324,1532.0,1457,FL,904,N927AT,87.0,93.0,73.0,35.0,41.0,ATL,PHF,508,2,12,0,NaN,0,35,0,0,0,0
5070842,2006,9,12,2,2125.0,2130,2241.0,2250,F9,589,N949FR,76.0,80.0,64.0,-9.0,-5.0,DEN,SLC,391,3,9,0,NaN,0,0,0,0,0,0


In [31]:
flights_clean.drop(index = flights_clean[flights_clean.duplicated()].index, inplace = True)

In [32]:
flights_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [33]:
flights_clean.duplicated().sum()

0

In [34]:
flights_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141915 entries, 0 to 7141914
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             int64  
 20  TaxiOut            int64  
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       int64  
 25  WeatherDelay      

#### **Issue**: There are missing values in these columns in the `flights` dataframe: `DepTime`, `ArrTime`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`, `ArrDelay`, `DepDelay` and `CancellationCode`.

##### **Define**
- For each column, inspect sample records with missing values.
- Identify why values are missing.
- Determine whether values should be replaced, or whether entries should be dropped or left as is.
- If any values are dropped, reindex dataframe.

##### **Code**
I will inspect some of the flights for which there are missing values in the `DepTime` column.

In [35]:
flights_clean.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,N657AW,281.0,273.0,223.0,6.0,-2.0,ATL,PHX,1587,45,13,0,NaN,0,0,0,0,0,0
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,N834AW,260.0,265.0,214.0,-5.0,0.0,ATL,PHX,1587,27,19,0,NaN,0,0,0,0,0,0
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,N605AW,235.0,258.0,220.0,-23.0,0.0,ATL,PHX,1587,4,11,0,NaN,0,0,0,0,0,0
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,N312AW,152.0,158.0,126.0,-8.0,-2.0,AUS,PHX,872,16,10,0,NaN,0,0,0,0,0,0
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,N309AW,171.0,163.0,132.0,0.0,-8.0,AUS,PHX,872,27,12,0,NaN,0,0,0,0,0,0


In [36]:
flights_clean[flights_clean["DepTime"].isnull()].sample(15)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
700747,2006,2,12,7,NaN,620,NaN,716,YV,7272,N852MJ,NaN,56.0,NaN,NaN,NaN,ORF,IAD,157,0,0,1,B,0,0,0,0,0,0
2573503,2006,5,17,3,NaN,1950,NaN,2100,UA,631,000000,NaN,130.0,NaN,NaN,NaN,DCA,ORD,612,0,0,1,A,0,0,0,0,0,0
2030565,2006,4,16,7,NaN,644,NaN,939,EV,4579,N930EV,NaN,115.0,NaN,NaN,NaN,SHV,CVG,686,0,0,1,A,0,0,0,0,0,0
2883665,2006,5,14,7,NaN,1334,NaN,1355,DL,1095,N984DL,NaN,81.0,NaN,NaN,NaN,ATL,GPT,352,0,0,1,A,0,0,0,0,0,0
623947,2006,2,6,1,NaN,1455,NaN,1600,WN,1460,0,NaN,65.0,NaN,NaN,NaN,BUR,SJC,296,0,0,1,A,0,0,0,0,0,0
6098018,2006,11,16,4,NaN,1830,NaN,1947,OH,5283,0,NaN,77.0,NaN,NaN,NaN,JFK,DCA,213,0,0,1,B,0,0,0,0,0,0
6648297,2006,12,15,5,NaN,815,NaN,1219,XE,2921,0,NaN,184.0,NaN,NaN,NaN,IAH,IAD,1190,0,0,1,B,0,0,0,0,0,0
811480,2006,2,11,6,NaN,2055,NaN,2347,UA,470,000000,NaN,112.0,NaN,NaN,NaN,ORD,PHL,678,0,0,1,B,0,0,0,0,0,0
3341183,2006,6,21,3,NaN,1703,NaN,1756,NW,1782,N764NC,NaN,53.0,NaN,NaN,NaN,GRR,DTW,120,0,0,1,A,0,0,0,0,0,0
1479524,2006,3,19,7,NaN,938,NaN,1043,MQ,3309,0,NaN,65.0,NaN,NaN,NaN,DFW,LAW,140,0,0,1,A,0,0,0,0,0,0


One thing all the rows above in common have is a value of 1 in the `Cancelled` column. I will now verify whether all missing departure time values correspond to cancelled flights only.

In [37]:
flights_clean[flights_clean["DepTime"].isnull()]["Cancelled"].value_counts()

1    121934
Name: Cancelled, dtype: int64

This is the case. None of the rows with missing departure times will need to be cleaned, as some analysis will be conducted for these rows. I will now confirm whether this is the total number of all cancelled flights.

In [38]:
flights_clean["Cancelled"].value_counts()

0    7019981
1     121934
Name: Cancelled, dtype: int64

This is the case. There aren't any cancelled flights that incorrectly have a departure time recorded.

I will now turn to the missing values in the `ArrTime` column. Some of the missing arrival times could be explained by there being cancelled flights as I found above. However, there are more missing values in the arrival time column than there are missing values in the departure time column. I will inspect a sample of rows that have these missing arrival time values but do not have missing departure time values. 

In [39]:
flights_clean[(flights_clean["ArrTime"].isnull()) & (flights_clean["DepTime"].notnull())].sample(15)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
6465470,2006,11,16,4,2011.0,1755,NaN,2150,B6,173,N588JB,NaN,415.0,NaN,NaN,136.0,JFK,SJC,2569,0,74,0,NaN,1,0,0,0,0,0
5063142,2006,9,16,6,2230.0,2230,NaN,2355,EV,4701,N838AS,NaN,145.0,NaN,NaN,0.0,ATL,OMA,821,0,5,0,NaN,1,0,0,0,0,0
3439678,2006,6,22,4,1157.0,1200,NaN,1452,CO,1833,N13665,NaN,232.0,NaN,NaN,-3.0,LGA,IAH,1416,0,27,0,NaN,1,0,0,0,0,0
1970995,2006,4,6,4,1656.0,1505,NaN,1758,UA,364,N814UA,NaN,113.0,NaN,NaN,111.0,DEN,MSP,680,0,13,0,NaN,1,0,0,0,0,0
4729562,2006,8,13,7,1904.0,1906,NaN,2012,DL,776,N985DL,NaN,126.0,NaN,NaN,-2.0,ATL,MCI,692,0,15,0,NaN,1,0,0,0,0,0
2498259,2006,5,22,1,1937.0,1940,NaN,2113,OO,3911,N912EV,NaN,93.0,NaN,NaN,-3.0,SLC,DEN,391,0,22,0,NaN,1,0,0,0,0,0
2035651,2006,4,10,1,700.0,700,NaN,1027,EV,4814,N925EV,NaN,147.0,NaN,NaN,0.0,GRB,ATL,774,0,15,0,NaN,1,0,0,0,0,0
4868307,2006,9,23,6,1353.0,1400,NaN,1647,XE,2120,N17159,NaN,167.0,NaN,NaN,-7.0,MKE,IAH,984,0,17,0,NaN,1,0,0,0,0,0
1837321,2006,4,30,7,1243.0,1225,NaN,1230,WN,1887,N399WN,NaN,65.0,NaN,NaN,18.0,DTW,MDW,229,0,10,0,NaN,1,0,0,0,0,0
3451248,2006,6,13,2,1356.0,1345,NaN,1626,CO,23,N29124,NaN,221.0,NaN,NaN,11.0,EWR,IAH,1400,0,34,0,NaN,1,0,0,0,0,0


A commonality for all the sample rows above is that they have a value of 1 in the `Diverted` column. I will now verify whether all missing arrival times, where departure times are present, correspond only to diverted flights.

In [40]:
flights_clean[(flights_clean["ArrTime"].isnull()) & (flights_clean["DepTime"].notnull())]["Diverted"].value_counts()

1    16185
Name: Diverted, dtype: int64

This is the case. Thus, none of the missing values in the departure time column will need to be cleaned, as some analysis will be conducted for these rows. I will now check whether this is the total number of all diverted flights.

In [41]:
flights_clean["Diverted"].value_counts()

0    7125730
1      16185
Name: Diverted, dtype: int64

This is the case. There aren't any diverted flights that erroneously have arrival times recorded.

I will now confirm that all missing values for the `ActualElapsedTime` column correspond only to flights that were either cancelled or diverted.

In [42]:
flights_clean[flights_clean["ActualElapsedTime"].isnull()][["Cancelled", "Diverted"]].value_counts()

Cancelled  Diverted
1          0           121934
0          1            16185
dtype: int64

These values are equal to the total values for cancelled and diverted flights, so this is satisfactory.

There are 4 flights for which there missing values for the expected flight length, in the `CRSElapsedTime` column. I will now inspect these rows.

In [43]:
flights_clean[flights_clean["CRSElapsedTime"].isnull()]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
4898102,2006,9,28,4,NaN,0,NaN,0,YV,2729,N27318,NaN,NaN,NaN,NaN,NaN,LAS,SBA,288,0,0,1,C,0,0,0,0,0,0
4898429,2006,9,28,4,NaN,0,NaN,0,YV,2738,N27318,NaN,NaN,NaN,NaN,NaN,SBA,LAS,288,0,0,1,C,0,0,0,0,0,0
4898976,2006,9,29,5,NaN,0,NaN,0,YV,2729,N27318,NaN,NaN,NaN,NaN,NaN,LAS,SBA,288,0,0,1,C,0,0,0,0,0,0
4899303,2006,9,29,5,NaN,0,NaN,0,YV,2738,N27318,NaN,NaN,NaN,NaN,NaN,SBA,LAS,288,0,0,1,C,0,0,0,0,0,0


These flights are all flights that were cancelled. However, they differ from the other cancelled flights we viewed above in that they do not have a scheduled departure time. In fact, for all 4 of these flights, the only other recorded information is the date, carrier, flight number, tail number, origin and destination airports, and the distance of the flight. These rows likely need to be dropped from the dataframe. I will now confirm whether these 4 rows are the only instances where the value of the scheduled departure time is 0.

In [44]:
len(flights_clean[flights_clean["CRSDepTime"] == 0])

4

This is the case. For completeness, I will also now verify that these are the only rows for which the value of the scheduled arrival time is 0.

In [45]:
len(flights_clean[flights_clean["CRSArrTime"] == 0])

5

There seems to be one more row with a scheduled arrival time of 0. I will inspect all 5 rows.

In [46]:
flights_clean[flights_clean["CRSArrTime"] == 0]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
4898102,2006,9,28,4,NaN,0,NaN,0,YV,2729,N27318,NaN,NaN,NaN,NaN,NaN,LAS,SBA,288,0,0,1,C,0,0,0,0,0,0
4898429,2006,9,28,4,NaN,0,NaN,0,YV,2738,N27318,NaN,NaN,NaN,NaN,NaN,SBA,LAS,288,0,0,1,C,0,0,0,0,0,0
4898976,2006,9,29,5,NaN,0,NaN,0,YV,2729,N27318,NaN,NaN,NaN,NaN,NaN,LAS,SBA,288,0,0,1,C,0,0,0,0,0,0
4899303,2006,9,29,5,NaN,0,NaN,0,YV,2738,N27318,NaN,NaN,NaN,NaN,NaN,SBA,LAS,288,0,0,1,C,0,0,0,0,0,0
6892672,2006,12,17,7,2233.0,2315,2334.0,0,HA,26,N589HA,61.0,45.0,32.0,-26.0,-42.0,PDX,SEA,129,20,9,0,NaN,0,0,0,0,0,0


The arrival time for the bottom row shows as 23:34, and the arrival delay recorded is -26. This means the scheduled arrival time for the flight was midnight. Due to the misrepresentation of the departure and arrival times in the dataframe as integers, 00:00 for midnight would show as the integer 0, but the value is valid and correct. Therefore, the 5th row does not to be dropped with the other 4.

In [47]:
# CODE HERE TO DROP ROWS
flights_clean.drop(flights_clean[flights_clean["CRSElapsedTime"].isnull()].index, axis = "index", inplace = True)

In [48]:
flights_clean.reset_index(drop = True, inplace = True)

The total number of cancelled flights will now be lower by 4.

I will now confirm that all missing values for the `AirTime` and `ArrDelay` columns correspond only to flights that were either cancelled or diverted.

In [49]:
flights_clean[flights_clean["AirTime"].isnull()][["Cancelled", "Diverted"]].value_counts()

Cancelled  Diverted
1          0           121930
0          1            16185
dtype: int64

In [50]:
flights_clean[flights_clean["ArrDelay"].isnull()][["Cancelled", "Diverted"]].value_counts()

Cancelled  Diverted
1          0           121930
0          1            16185
dtype: int64

These values are equal to the total values for cancelled and diverted flights, so this is satisfactory.

I will now confirm that all missing values for the `DepDelay` column correspond only to flights that were cancelled.

In [51]:
flights_clean[flights_clean["DepDelay"].isnull()]["Cancelled"].value_counts()

1    121930
Name: Cancelled, dtype: int64

This value is equal to the total value for cancelled flights, so this is satisfactory.

Null values are expected in the `CancellationCode` column, since there are flights that were not cancelled. So the null values are not an error and do not need to be cleaned. I will verify that the number of values in this column is the same as the number of cancelled flights.

In [52]:
flights_clean["CancellationCode"].notnull().value_counts()

False    7019981
True      121930
Name: CancellationCode, dtype: int64

This is satisfactory.

##### **Test**

In [53]:
flights_clean.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141911 entries, 0 to 7141910
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               7141911 non-null  int64  
 1   Month              7141911 non-null  int64  
 2   DayofMonth         7141911 non-null  int64  
 3   DayOfWeek          7141911 non-null  int64  
 4   DepTime            7019981 non-null  float64
 5   CRSDepTime         7141911 non-null  int64  
 6   ArrTime            7003796 non-null  float64
 7   CRSArrTime         7141911 non-null  int64  
 8   UniqueCarrier      7141911 non-null  object 
 9   FlightNum          7141911 non-null  int64  
 10  TailNum            7141911 non-null  object 
 11  ActualElapsedTime  7003796 non-null  float64
 12  CRSElapsedTime     7141911 non-null  float64
 13  AirTime            7003796 non-null  float64
 14  ArrDelay           7003796 non-null  float64
 15  DepDelay           7019981 non-n

#### **Issue**: The `Year`, `Month` and `DayofMonth` columns are different parts of the same variable, the date.
To adhere to the principles of data tidiness, the date variable needs to be stored in only one column. This would involve appropriately concatenating the values in the 3 different columns and storing the result in a new column. The result from this operation, though, would not be in the correct format or have the correct data type. However, since this concatenation is an intermediary step when addressing the next issue with my data, for efficiency, this issue will be fixed in the next subsection, to avoid storing an incompletely processed form of the date.

#### **Issue**: The columns `DepTime`, `CRSDepTime`, `ArrTime`, and`CRSArrTime` have the *int* and *float* data types, which is incorrect.
The values in these columns are not numerical data, but times, and therefore shold be stored as such. The timestamp is the most appropriate data type for storing the dates from the columns mentioned in the previous subsection and the times in the columns mentioned in this subsection, combined. Therefore, for each of these 4 columns, the values that make up the date need to be concatenated with the values that make up the time.

##### **Define**
- Identify subset of rows where conversion of data types will not throw errors and will produce a valid result.
- Convert values of year, month, day and time values for each column into strings.
- Concatenate strings.
- Convert strings to timestamps

##### **Code**
Not all rows in the raw data have values that can be successfully converted to timestamps. Some of the rows have null values, and some of the rows hold integer values that are larger than 2359, which is the maximum value for time. Both of these issues will be addressed later elsewhere in this notebook. As such, for this subsection, I will work only with the rows that have valid values in all the columns that are being cleaned. I will make use of masks to select these subsets.

In [54]:
# Identify rows where times are not null and are not greater than 2359
DepTime_2359_mask = (flights_clean["DepTime"] < 2400) & (flights_clean["DepTime"].notnull())
DepTime_2359_mask.value_counts()

True     7018579
False     123332
Name: DepTime, dtype: int64

In [55]:
flights_clean.loc[DepTime_2359_mask, "ActualDepTime"] = pd.to_datetime(flights_clean.loc[DepTime_2359_mask, :]["Year"].apply(str) + flights_clean.loc[DepTime_2359_mask, :]["Month"].apply(str).str.zfill(2) + flights_clean.loc[DepTime_2359_mask, :]["DayofMonth"].apply(str).str.zfill(2) + flights_clean.loc[DepTime_2359_mask, :]["DepTime"].apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M")

In [56]:
# Identify rows where times are not null and are not greater than 2359
ArrTime_2359_mask = (flights_clean["ArrTime"] < 2400) & (flights_clean["ArrTime"].notnull())
ArrTime_2359_mask.value_counts()

True     6997887
False     144024
Name: ArrTime, dtype: int64

In [57]:
flights_clean.loc[ArrTime_2359_mask, "ActualArrTime"] = pd.to_datetime(flights_clean.loc[ArrTime_2359_mask, :]["Year"].apply(str) + flights_clean.loc[ArrTime_2359_mask, :]["Month"].apply(str).str.zfill(2) + flights_clean.loc[ArrTime_2359_mask, :]["DayofMonth"].apply(str).str.zfill(2) + flights_clean.loc[ArrTime_2359_mask, :]["ArrTime"].apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M")

For the `CRSDepTime` and `CRSArrTime` columns, there are no null values, and the values are all between 0 and 2359, inclusive of both values. Therefore, no masks are necessary for either column. The conversion can be implemented on the entire column directly.

In [58]:
flights_clean.loc[:, "SchedDepTime"] = pd.to_datetime(flights_clean["Year"].apply(str) + flights_clean["Month"].apply(str).str.zfill(2) + flights_clean["DayofMonth"].apply(str).str.zfill(2) + flights_clean["CRSDepTime"].apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M")

In [59]:
flights_clean.loc[:, "SchedArrTime"] = pd.to_datetime(flights_clean["Year"].apply(str) + flights_clean["Month"].apply(str).str.zfill(2) + flights_clean["DayofMonth"].apply(str).str.zfill(2) + flights_clean["CRSArrTime"].apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M")

##### **Test**

In [60]:
flights_clean[["Year", "Month", "DayofMonth", "DepTime", "CRSDepTime", "ArrTime", "CRSArrTime",
               "ActualDepTime", "SchedDepTime", "ActualArrTime", "SchedArrTime"]].sample(5)

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualDepTime,SchedDepTime,ActualArrTime,SchedArrTime
259121,2006,1,11,930.0,835,1045.0,1000,2006-01-11 09:30:00,2006-01-11 08:35:00,2006-01-11 10:45:00,2006-01-11 10:00:00
5591326,2006,10,21,2010.0,1950,2154.0,2130,2006-10-21 20:10:00,2006-10-21 19:50:00,2006-10-21 21:54:00,2006-10-21 21:30:00
705242,2006,2,18,1936.0,1730,2121.0,1928,2006-02-18 19:36:00,2006-02-18 17:30:00,2006-02-18 21:21:00,2006-02-18 19:28:00
377317,2006,1,11,1652.0,1655,1751.0,1755,2006-01-11 16:52:00,2006-01-11 16:55:00,2006-01-11 17:51:00,2006-01-11 17:55:00
4921134,2006,9,7,2201.0,2133,2230.0,2158,2006-09-07 22:01:00,2006-09-07 21:33:00,2006-09-07 22:30:00,2006-09-07 21:58:00


In [61]:
len(flights_clean[(flights_clean["SchedArrTime"] > flights_clean["SchedDepTime"]) & ((flights_clean["SchedArrTime"] - flights_clean["SchedDepTime"]) > pd.to_timedelta(18, "h"))])

0

In [62]:
len(flights_clean[(flights_clean["SchedDepTime"] > flights_clean["SchedArrTime"]) & ((flights_clean["SchedDepTime"] - flights_clean["SchedArrTime"]) > pd.to_timedelta(flights_clean["CRSElapsedTime"], unit = "m"))])

140035

#### **Issue**: The scheduled departure times are not always on the same day as the actual departure times.

##### **Define**
- Identify rows where scheduled departure time is not on the same day as actual departure time.
- For each row, either add or subtract 1 day to the timestamp, as applicable.
- Amend timestamps for scheduled arrival time column to match new date in scheduled departure column.

##### **Code**

In [63]:
DepTime_Ahead_Mask = (flights_clean["SchedDepTime"] > flights_clean["ActualDepTime"]) & \
((flights_clean["SchedDepTime"] - flights_clean["ActualDepTime"]) > pd.to_timedelta(abs(flights_clean["DepDelay"]), "m"))
DepTime_Ahead_Mask.value_counts()

False    7124705
True       17206
dtype: int64

In [64]:
flights_clean.loc[DepTime_Ahead_Mask, "SchedDepTime"] -= pd.Timedelta(days = 1)

In [65]:
DepTime_Behind_Mask = (flights_clean["ActualDepTime"] > flights_clean["SchedDepTime"]) & \
((flights_clean["ActualDepTime"] - flights_clean["SchedDepTime"]) > pd.to_timedelta(abs(flights_clean["DepDelay"]), "m"))
DepTime_Behind_Mask.value_counts()

False    7141844
True          67
dtype: int64

In [66]:
flights_clean.loc[DepTime_Behind_Mask, "SchedDepTime"] += pd.Timedelta(days = 1)

In [67]:
flights_clean["SchedArrTime"] = pd.to_datetime(flights_clean["SchedDepTime"].dt.date.astype(str)
                                               + " " + flights_clean["SchedArrTime"].dt.time.astype(str))

##### **Test**

In [68]:
flights_clean[DepTime_Ahead_Mask][["Year", "Month", "DayofMonth", "DepTime", "CRSDepTime",
                                   "ArrTime", "CRSArrTime", "ActualDepTime", "SchedDepTime",
                                   "ActualArrTime", "SchedArrTime"]].sample(8)

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualDepTime,SchedDepTime,ActualArrTime,SchedArrTime
276354,2006,1,1,5.0,2355,233.0,222,2006-01-01 00:05:00,2005-12-31 23:55:00,2006-01-01 02:33:00,2005-12-31 02:22:00
2100838,2006,4,2,42.0,2202,123.0,2246,2006-04-02 00:42:00,2006-04-01 22:02:00,2006-04-02 01:23:00,2006-04-01 22:46:00
5287667,2006,9,28,16.0,2340,423.0,330,2006-09-28 00:16:00,2006-09-27 23:40:00,2006-09-28 04:23:00,2006-09-27 03:30:00
3549668,2006,7,18,134.0,2000,223.0,2100,2006-07-18 01:34:00,2006-07-17 20:00:00,2006-07-18 02:23:00,2006-07-17 21:00:00
3379375,2006,6,22,42.0,2355,634.0,530,2006-06-22 00:42:00,2006-06-21 23:55:00,2006-06-22 06:34:00,2006-06-21 05:30:00
2674357,2006,5,29,725.0,1940,830.0,2049,2006-05-29 07:25:00,2006-05-28 19:40:00,2006-05-29 08:30:00,2006-05-28 20:49:00
707100,2006,2,20,25.0,2356,230.0,214,2006-02-20 00:25:00,2006-02-19 23:56:00,2006-02-20 02:30:00,2006-02-19 02:14:00
4619868,2006,8,20,10.0,2140,142.0,2320,2006-08-20 00:10:00,2006-08-19 21:40:00,2006-08-20 01:42:00,2006-08-19 23:20:00


In [69]:
flights_clean[DepTime_Behind_Mask][["Year", "Month", "DayofMonth", "DepTime", "CRSDepTime",
                                    "ArrTime", "CRSArrTime", "ActualDepTime", "SchedDepTime",
                                    "ActualArrTime", "SchedArrTime"]].sample(8)

,Year,Month,DayofMonth,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualDepTime,SchedDepTime,ActualArrTime,SchedArrTime
4038426,2006,7,2,2355.0,5,654.0,711,2006-07-02 23:55:00,2006-07-03 00:05:00,2006-07-02 06:54:00,2006-07-03 07:11:00
2219088,2006,4,13,1341.0,1415,1740.0,1725,2006-04-13 13:41:00,2006-04-13 14:15:00,2006-04-13 17:40:00,2006-04-13 17:25:00
4554135,2006,8,24,2358.0,10,742.0,734,2006-08-24 23:58:00,2006-08-25 00:10:00,2006-08-24 07:42:00,2006-08-25 07:34:00
2715092,2006,5,9,2358.0,10,720.0,733,2006-05-09 23:58:00,2006-05-10 00:10:00,2006-05-09 07:20:00,2006-05-10 07:33:00
4688706,2006,8,27,2356.0,15,522.0,546,2006-08-27 23:56:00,2006-08-28 00:15:00,2006-08-27 05:22:00,2006-08-28 05:46:00
2710650,2006,5,15,2356.0,10,519.0,526,2006-05-15 23:56:00,2006-05-16 00:10:00,2006-05-15 05:19:00,2006-05-16 05:26:00
4657460,2006,8,13,2357.0,5,702.0,711,2006-08-13 23:57:00,2006-08-14 00:05:00,2006-08-13 07:02:00,2006-08-14 07:11:00
4663929,2006,8,16,2358.0,5,655.0,711,2006-08-16 23:58:00,2006-08-17 00:05:00,2006-08-16 06:55:00,2006-08-17 07:11:00


In [70]:
flights_clean[DepTime_Ahead_Mask | DepTime_Behind_Mask][["Year", "Month", "DayofMonth", "DepTime", "CRSDepTime", "ActualDepTime", "SchedDepTime"]].sample(8)

,Year,Month,DayofMonth,DepTime,CRSDepTime,ActualDepTime,SchedDepTime
6937011,2006,12,12,14.0,2245,2006-12-12 00:14:00,2006-12-11 22:45:00
3503386,2006,6,28,140.0,2345,2006-06-28 01:40:00,2006-06-27 23:45:00
5310497,2006,9,10,1.0,2330,2006-09-10 00:01:00,2006-09-09 23:30:00
4580263,2006,8,31,701.0,2055,2006-08-31 07:01:00,2006-08-30 20:55:00
7131387,2006,12,21,120.0,2332,2006-12-21 01:20:00,2006-12-20 23:32:00
1253889,2006,3,20,108.0,2355,2006-03-20 01:08:00,2006-03-19 23:55:00
541707,2006,1,2,19.0,2245,2006-01-02 00:19:00,2006-01-01 22:45:00
1739216,2006,4,9,19.0,2359,2006-04-09 00:19:00,2006-04-08 23:59:00


#### **Issue**: The origin airports are often not in the same timezone as the destination airports.
Only one "flight date" is recorded for each row in the original data, presumably to be valid for all time columns in the row. But for some entries this date is correct only for the departure times. For some timezone differences or flight durations, and for some combinations of the two, the date at the destination airport is not the same as the date at the origin airport. Since date and time are now stored together as timestamps, these date differences need to be reflected in some of the timestamps that have now been created in each of the columns `ActualDepTime`, `SchedDepTime`, `ActualArrTime` and `SchedArrTime`.

##### **Define**
- For each column, identify rows where the date is incorrect.
- Correct each date by adding or subtracting 1 day as necessary.

##### **Code**

In [71]:
successful_flights_mask = (flights_clean["Diverted"] == 0) & (flights_clean["Cancelled"] == 0)
successful_flights_mask.value_counts()

True     7003796
False     138115
dtype: int64

When actual departure time is on one day, and actual arrival time is on the next day.

In [72]:
# ActualDepTime
DepTime_Timezone_Mask = (flights_clean["ActualDepTime"] > flights_clean["ActualArrTime"]) & \
((flights_clean["ActualDepTime"] - flights_clean["ActualArrTime"]) > pd.to_timedelta(flights_clean["ActualElapsedTime"], unit = "m"))
DepTime_Timezone_Mask.value_counts()

False    6956951
True      184960
dtype: int64

In [73]:
flights_clean[successful_flights_mask & DepTime_Timezone_Mask].sample(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
2331092,2006,5,22,1,2225.0,2230,700.0,737,US,925,N906AW,335.0,367.0,316.0,-37.0,-5.0,HNL,PHX,2917,3,16,0,NaN,0,0,0,0,0,0,2006-05-22 22:25:00,2006-05-22 07:00:00,2006-05-22 22:30:00,2006-05-22 07:37:00
3435779,2006,6,24,6,2202.0,2155,611.0,605,B6,186,N579JB,309.0,310.0,464.0,6.0,7.0,SAN,JFK,2446,9,16,0,NaN,0,0,0,0,0,0,2006-06-24 22:02:00,2006-06-24 06:11:00,2006-06-24 21:55:00,2006-06-24 06:05:00
5553689,2006,10,13,5,2029.0,2035,449.0,453,UA,68,N548UA,320.0,318.0,305.0,-4.0,-6.0,LIH,LAX,2615,3,12,0,NaN,0,0,0,0,0,0,2006-10-13 20:29:00,2006-10-13 04:49:00,2006-10-13 20:35:00,2006-10-13 04:53:00
6216972,2006,11,27,1,2318.0,2245,712.0,622,US,1548,N110UW,294.0,277.0,272.0,50.0,33.0,SEA,CLT,2279,7,15,0,NaN,0,33,0,17,0,0,2006-11-27 23:18:00,2006-11-27 07:12:00,2006-11-27 22:45:00,2006-11-27 06:22:00
6392919,2006,11,20,1,2223.0,2230,521.0,525,AA,246,N5EXAA,298.0,295.0,275.0,-4.0,-7.0,KOA,LAX,2504,14,9,0,NaN,0,0,0,0,0,0,2006-11-20 22:23:00,2006-11-20 05:21:00,2006-11-20 22:30:00,2006-11-20 05:25:00


In [74]:
flights_clean.loc[successful_flights_mask & DepTime_Timezone_Mask, "ActualArrTime"] += pd.Timedelta(days = 1)

When scheduled departure time is on one day, and scheduled arrival time is on the next day.

In [75]:
# SchedDepTime
SchedDepTime_Timezone_Mask = (flights_clean["SchedDepTime"] > flights_clean["SchedArrTime"]) & \
((flights_clean["SchedDepTime"] - flights_clean["SchedArrTime"]) > pd.to_timedelta(flights_clean["CRSElapsedTime"], unit = "m"))
SchedDepTime_Timezone_Mask.value_counts()

False    7001876
True      140035
dtype: int64

In [76]:
flights_clean[successful_flights_mask & SchedDepTime_Timezone_Mask].sample(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
1637732,2006,3,7,2,2109.0,2110,38.0,20,B6,452,N547JB,209.0,190.0,190.0,18.0,-1.0,FLL,BOS,1237,3,16,0,NaN,0,0,0,18,0,0,2006-03-07 21:09:00,2006-03-08 00:38:00,2006-03-07 21:10:00,2006-03-07 00:20:00
1742568,2006,4,15,6,2152.0,2152,506.0,525,US,18,N818AW,254.0,273.0,241.0,-19.0,0.0,PHX,JFK,2153,5,8,0,NaN,0,0,0,0,0,0,2006-04-15 21:52:00,2006-04-16 05:06:00,2006-04-15 21:52:00,2006-04-15 05:25:00
3429286,2006,6,8,4,2313.0,2315,528.0,530,B6,92,N623JB,255.0,255.0,353.0,-2.0,-2.0,SLC,JFK,1989,5,17,0,NaN,0,0,0,0,0,0,2006-06-08 23:13:00,2006-06-09 05:28:00,2006-06-08 23:15:00,2006-06-08 05:30:00
5774220,2006,10,10,2,1748.0,1800,50.0,55,NW,1198,N363NB,242.0,235.0,216.0,-5.0,-12.0,LAS,DTW,1750,10,16,0,NaN,0,0,0,0,0,0,2006-10-10 17:48:00,2006-10-11 00:50:00,2006-10-10 18:00:00,2006-10-10 00:55:00
1080466,2006,2,4,6,2336.0,2330,642.0,656,DL,2052,N6703D,246.0,266.0,225.0,-14.0,6.0,LAS,FLL,2174,4,17,0,NaN,0,0,0,0,0,0,2006-02-04 23:36:00,2006-02-05 06:42:00,2006-02-04 23:30:00,2006-02-04 06:56:00


In [77]:
flights_clean.loc[successful_flights_mask & SchedDepTime_Timezone_Mask, "SchedArrTime"] += pd.Timedelta(days = 1)

When actual departure time is on one day and actual arrival time is on previous day.

In [78]:
# ActualArrTime
ActualArrTime_Timezone_Mask = (flights_clean["ActualArrTime"] > flights_clean["ActualDepTime"]) & \
((flights_clean["ActualArrTime"] - flights_clean["ActualDepTime"]) > pd.to_timedelta(18, "h"))
ActualArrTime_Timezone_Mask.value_counts()

False    7141843
True          68
dtype: int64

In [79]:
flights_clean[(successful_flights_mask & ActualArrTime_Timezone_Mask)].sample(8)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
2440230,2006,5,15,1,1013.0,1021,939.0,1029,YV,7258,N400MJ,26.0,68.0,10.0,-50.0,-8.0,MBS,ORD,222,9,7,0,NaN,0,0,0,0,0,0,2006-05-15 10:13:00,2006-05-16 09:39:00,2006-05-15 10:21:00,2006-05-15 10:29:00
6715325,2006,12,11,1,1701.0,1647,1625.0,1606,OO,5740,N560SW,24.0,19.0,17.0,19.0,14.0,YUM,IPL,58,2,5,0,NaN,0,0,0,0,0,19,2006-12-11 17:01:00,2006-12-12 16:25:00,2006-12-11 16:47:00,2006-12-12 16:06:00
6122874,2006,11,10,5,1713.0,1647,1641.0,1606,OO,5740,N250YV,28.0,19.0,15.0,35.0,26.0,YUM,IPL,58,4,9,0,NaN,0,0,0,0,0,35,2006-11-10 17:13:00,2006-11-11 16:41:00,2006-11-10 16:47:00,2006-11-11 16:06:00
6718418,2006,12,13,3,1640.0,1647,1607.0,1606,OO,5740,N393SW,27.0,19.0,18.0,1.0,-7.0,YUM,IPL,58,2,7,0,NaN,0,0,0,0,0,0,2006-12-13 16:40:00,2006-12-14 16:07:00,2006-12-13 16:47:00,2006-12-14 16:06:00
6124366,2006,11,11,6,1715.0,1647,1643.0,1606,OO,5740,N289YV,28.0,19.0,17.0,37.0,28.0,YUM,IPL,58,2,9,0,NaN,0,0,0,0,0,37,2006-11-11 17:15:00,2006-11-12 16:43:00,2006-11-11 16:47:00,2006-11-12 16:06:00
6730992,2006,12,21,4,1731.0,1651,1656.0,1610,OO,5740,N291SW,25.0,19.0,16.0,46.0,40.0,YUM,IPL,58,3,6,0,NaN,0,0,0,0,0,46,2006-12-21 17:31:00,2006-12-22 16:56:00,2006-12-21 16:51:00,2006-12-22 16:10:00
3638748,2006,7,9,7,1208.0,1009,1137.0,954,YV,7196,N17175,29.0,45.0,20.0,103.0,119.0,SBN,ORD,84,3,6,0,NaN,0,103,0,0,0,0,2006-07-09 12:08:00,2006-07-10 11:37:00,2006-07-09 10:09:00,2006-07-09 09:54:00
6729387,2006,12,20,3,1750.0,1651,1712.0,1610,OO,5740,N213SW,22.0,19.0,15.0,62.0,59.0,YUM,IPL,58,2,5,0,NaN,0,0,0,0,0,62,2006-12-20 17:50:00,2006-12-21 17:12:00,2006-12-20 16:51:00,2006-12-21 16:10:00


There are entries where the actual arrival time was erroneously increased by 1 day because the values for actual elapsed time are used in the calculation, and these flights were all very short. They are distinguishable from the other entries because their values in `DepTime` are bigger than the values for `ArrTime`.

In [80]:
(successful_flights_mask & ActualArrTime_Timezone_Mask & (flights_clean["DepTime"] > flights_clean["ArrTime"])).value_counts()

False    7141853
True          58
dtype: int64

I will now correct the actual arrival time dates for these entries by subtracting 1 day.

In [81]:
flights_clean.loc[successful_flights_mask & ActualArrTime_Timezone_Mask & \
 (flights_clean["DepTime"] > flights_clean["ArrTime"]), "ActualArrTime"] -= pd.Timedelta(days = 1)

I will now compute the mask afresh.

In [82]:
# ActualArrTime
ActualArrTime_Timezone_Mask = (flights_clean["ActualArrTime"] > flights_clean["ActualDepTime"]) & \
((flights_clean["ActualArrTime"] - flights_clean["ActualDepTime"]) > pd.to_timedelta(18, "h"))
ActualArrTime_Timezone_Mask.value_counts()

False    7141901
True          10
dtype: int64

I will now view all the rows since there are only 10 of them.

In [83]:
flights_clean[successful_flights_mask & ActualArrTime_Timezone_Mask]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
560286,2006,1,15,7,3.0,2256,2358.0,2303,DL,1469,N903DL,55.0,67.0,42.0,55.0,67.0,ATL,PNS,272,4,9,0,NaN,0,55,0,0,0,0,2006-01-15 00:03:00,2006-01-15 23:58:00,2006-01-14 22:56:00,2006-01-14 23:03:00
2874252,2006,5,7,7,17.0,2305,2359.0,2258,DL,1130,N904DL,42.0,53.0,27.0,61.0,72.0,ATL,BHM,134,3,12,0,NaN,0,0,4,0,0,57,2006-05-07 00:17:00,2006-05-07 23:59:00,2006-05-06 23:05:00,2006-05-06 22:58:00
3498419,2006,6,25,7,10.0,2035,2350.0,2030,DL,947,N905DL,40.0,55.0,26.0,200.0,215.0,ATL,BHM,134,5,9,0,NaN,0,26,0,0,0,174,2006-06-25 00:10:00,2006-06-25 23:50:00,2006-06-24 20:35:00,2006-06-24 20:30:00
4115757,2006,7,23,7,2.0,2305,2338.0,2300,DL,1130,N903DE,36.0,55.0,24.0,38.0,57.0,ATL,BHM,134,3,9,0,NaN,0,1,0,0,0,37,2006-07-23 00:02:00,2006-07-23 23:38:00,2006-07-22 23:05:00,2006-07-22 23:00:00
4333561,2006,8,24,4,1.0,2053,2357.0,2100,OO,6953,N939SW,56.0,67.0,44.0,177.0,188.0,TVC,ORD,224,5,7,0,NaN,0,0,0,0,0,177,2006-08-24 00:01:00,2006-08-24 23:57:00,2006-08-23 20:53:00,2006-08-23 21:00:00
4722962,2006,8,8,2,9.0,2305,2350.0,2300,DL,1130,N969DL,41.0,55.0,21.0,50.0,64.0,ATL,BHM,134,6,14,0,NaN,0,0,0,0,0,50,2006-08-08 00:09:00,2006-08-08 23:50:00,2006-08-07 23:05:00,2006-08-07 23:00:00
4739629,2006,8,20,7,3.0,2305,2353.0,2300,DL,1130,N958DL,50.0,55.0,22.0,53.0,58.0,ATL,BHM,134,4,24,0,NaN,0,0,26,0,0,27,2006-08-20 00:03:00,2006-08-20 23:53:00,2006-08-19 23:05:00,2006-08-19 23:00:00
5314650,2006,9,13,3,11.0,2302,2359.0,2255,DL,794,N995DL,48.0,53.0,27.0,64.0,69.0,ATL,BHM,134,6,15,0,NaN,0,22,0,0,0,42,2006-09-13 00:11:00,2006-09-13 23:59:00,2006-09-12 23:02:00,2006-09-12 22:55:00
6498563,2006,11,1,3,2.0,2300,2348.0,2252,DL,794,N913DL,46.0,52.0,27.0,56.0,62.0,ATL,BHM,134,5,14,0,NaN,0,56,0,0,0,0,2006-11-01 00:02:00,2006-11-01 23:48:00,2006-10-31 23:00:00,2006-10-31 22:52:00
6929660,2006,12,3,7,4.0,2000,2354.0,2015,MQ,4296,N507AE,50.0,75.0,36.0,219.0,244.0,IND,ORD,177,5,9,0,NaN,0,171,0,0,0,48,2006-12-03 00:04:00,2006-12-03 23:54:00,2006-12-02 20:00:00,2006-12-02 20:15:00


The `DepTime` and `ArrTime` columns show values that meet the criteria. I will now clean the rows by subtracting 1 day from the `ActualArrTime` column.

In [84]:
flights_clean.loc[successful_flights_mask & ActualArrTime_Timezone_Mask, "ActualArrTime"] -= pd.Timedelta(days = 1)

In [85]:
# SchedArrTime
SchedArrTime_Timezone_Mask = (flights_clean["SchedArrTime"] > flights_clean["SchedDepTime"]) & \
((flights_clean["SchedArrTime"] - flights_clean["SchedDepTime"]) > pd.to_timedelta(18, "h"))
SchedArrTime_Timezone_Mask.value_counts()

False    7141798
True         113
dtype: int64

In [86]:
flights_clean[successful_flights_mask & SchedArrTime_Timezone_Mask].sample(8)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
6724696,2006,12,17,7,643.0,633,614.0,552,OO,5780,N233SW,31.0,19.0,17.0,22.0,10.0,YUM,IPL,58,4,10,0,NaN,0,22,0,0,0,0,2006-12-17 06:43:00,2006-12-17 06:14:00,2006-12-17 06:33:00,2006-12-18 05:52:00
6130322,2006,11,15,3,1659.0,1647,1626.0,1606,OO,5740,N293SW,27.0,19.0,15.0,20.0,12.0,YUM,IPL,58,2,10,0,NaN,0,20,0,0,0,0,2006-11-15 16:59:00,2006-11-15 16:26:00,2006-11-15 16:47:00,2006-11-16 16:06:00
6740146,2006,12,27,3,1843.0,1651,1810.0,1610,OO,5740,N251YV,27.0,19.0,17.0,120.0,112.0,YUM,IPL,58,3,7,0,NaN,0,0,0,0,0,120,2006-12-27 18:43:00,2006-12-27 18:10:00,2006-12-27 16:51:00,2006-12-28 16:10:00
6131946,2006,11,16,4,627.0,633,556.0,552,OO,5780,N301YV,29.0,19.0,17.0,4.0,-6.0,YUM,IPL,58,3,9,0,NaN,0,0,0,0,0,0,2006-11-16 06:27:00,2006-11-16 05:56:00,2006-11-16 06:33:00,2006-11-17 05:52:00
6704683,2006,12,4,1,627.0,633,605.0,552,OO,5780,N220SW,38.0,19.0,15.0,13.0,-6.0,YUM,IPL,58,5,18,0,NaN,0,0,0,0,0,0,2006-12-04 06:27:00,2006-12-04 06:05:00,2006-12-04 06:33:00,2006-12-05 05:52:00
6152472,2006,11,30,4,1706.0,1647,1635.0,1606,OO,5740,N292SW,29.0,19.0,15.0,29.0,19.0,YUM,IPL,58,4,10,0,NaN,0,29,0,0,0,0,2006-11-30 17:06:00,2006-11-30 16:35:00,2006-11-30 16:47:00,2006-12-01 16:06:00
6124366,2006,11,11,6,1715.0,1647,1643.0,1606,OO,5740,N289YV,28.0,19.0,17.0,37.0,28.0,YUM,IPL,58,2,9,0,NaN,0,0,0,0,0,37,2006-11-11 17:15:00,2006-11-11 16:43:00,2006-11-11 16:47:00,2006-11-12 16:06:00
6700173,2006,12,1,5,1714.0,1647,1643.0,1606,OO,5740,N213SW,29.0,19.0,17.0,37.0,27.0,YUM,IPL,58,4,8,0,NaN,0,0,0,0,0,37,2006-12-01 17:14:00,2006-12-01 16:43:00,2006-12-01 16:47:00,2006-12-02 16:06:00


In [87]:
(successful_flights_mask & SchedArrTime_Timezone_Mask & (flights_clean["CRSDepTime"] < flights_clean["CRSArrTime"])).value_counts()

False    7141911
dtype: int64

There aren't any flights for which the scheduled arrival time is on the previous day from the scheduled departure time. This means, for all 113 rows from the mask above, the scheduled arrival times need to be corrected by subtracting 1 day from each one.

In [88]:
flights_clean.loc[successful_flights_mask & SchedArrTime_Timezone_Mask & \
              (flights_clean["CRSDepTime"] > flights_clean["CRSArrTime"]), "SchedArrTime"] -= pd.Timedelta(days = 1)

I will now compute the mask afresh, and check whether any values that satisfy the criteroa are still present in the dataframe.

In [89]:
SchedArrTime_Timezone_Mask = (flights_clean["SchedArrTime"] > flights_clean["SchedDepTime"]) & \
((flights_clean["SchedArrTime"] - flights_clean["SchedDepTime"]) > pd.to_timedelta(18, "h"))
SchedArrTime_Timezone_Mask.value_counts()

False    7141911
dtype: int64

This is satisfactory, and concludes the coding for this section. I move on to the testing.

##### **Test**

In [90]:
# ActualDepTime
flights_clean[successful_flights_mask & DepTime_Timezone_Mask][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                                "ActualElapsedTime", "ActualDepTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ArrTime,ActualElapsedTime,ActualDepTime,ActualArrTime
2005087,2006,4,12,2250.0,531.0,221.0,2006-04-12 22:50:00,2006-04-13 05:31:00
4731011,2006,8,14,2225.0,609.0,284.0,2006-08-14 22:25:00,2006-08-15 06:09:00
4082888,2006,7,7,2321.0,904.0,403.0,2006-07-07 23:21:00,2006-07-08 09:04:00
4041614,2006,7,26,2358.0,534.0,216.0,2006-07-26 23:58:00,2006-07-27 05:34:00
462265,2006,1,5,2303.0,502.0,239.0,2006-01-05 23:03:00,2006-01-06 05:02:00
60642,2006,1,11,2314.0,35.0,81.0,2006-01-11 23:14:00,2006-01-12 00:35:00
1048483,2006,2,20,2131.0,15.0,164.0,2006-02-20 21:31:00,2006-02-21 00:15:00
4148822,2006,8,10,2235.0,152.0,377.0,2006-08-10 22:35:00,2006-08-11 01:52:00
3178353,2006,6,25,2221.0,113.0,232.0,2006-06-25 22:21:00,2006-06-26 01:13:00
4059418,2006,7,25,2324.0,535.0,251.0,2006-07-25 23:24:00,2006-07-26 05:35:00


In [91]:
# SchedDepTime
flights_clean[successful_flights_mask & SchedDepTime_Timezone_Mask][["Year", "Month", "DayofMonth", "CRSDepTime", "CRSArrTime", \
                                                                     "CRSElapsedTime", "SchedDepTime", "SchedArrTime"]].sample(10)

,Year,Month,DayofMonth,CRSDepTime,CRSArrTime,CRSElapsedTime,SchedDepTime,SchedArrTime
2309843,2006,5,24,2355,723,268.0,2006-05-24 23:55:00,2006-05-25 07:23:00
6804107,2006,12,26,2244,456,252.0,2006-12-26 22:44:00,2006-12-27 04:56:00
5798121,2006,10,11,1655,455,420.0,2006-10-11 16:55:00,2006-10-12 04:55:00
1965326,2006,4,9,2235,705,330.0,2006-04-09 22:35:00,2006-04-10 07:05:00
2058474,2006,4,23,2255,33,98.0,2006-04-22 22:55:00,2006-04-23 00:33:00
6761294,2006,12,28,2300,603,303.0,2006-12-28 23:00:00,2006-12-29 06:03:00
333689,2006,1,20,2259,24,85.0,2006-01-20 22:59:00,2006-01-21 00:24:00
3482382,2006,6,13,2315,640,265.0,2006-06-13 23:15:00,2006-06-14 06:40:00
1363663,2006,3,15,2140,448,308.0,2006-03-15 21:40:00,2006-03-16 04:48:00
4675282,2006,8,9,2015,20,245.0,2006-08-09 20:15:00,2006-08-10 00:20:00


In [92]:
# ActualArrTime
flights_clean[(successful_flights_mask & ActualArrTime_Timezone_Mask)][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                                        "ActualElapsedTime", "ActualDepTime", "ActualArrTime"]]

,Year,Month,DayofMonth,DepTime,ArrTime,ActualElapsedTime,ActualDepTime,ActualArrTime
560286,2006,1,15,3.0,2358.0,55.0,2006-01-15 00:03:00,2006-01-14 23:58:00
2874252,2006,5,7,17.0,2359.0,42.0,2006-05-07 00:17:00,2006-05-06 23:59:00
3498419,2006,6,25,10.0,2350.0,40.0,2006-06-25 00:10:00,2006-06-24 23:50:00
4115757,2006,7,23,2.0,2338.0,36.0,2006-07-23 00:02:00,2006-07-22 23:38:00
4333561,2006,8,24,1.0,2357.0,56.0,2006-08-24 00:01:00,2006-08-23 23:57:00
4722962,2006,8,8,9.0,2350.0,41.0,2006-08-08 00:09:00,2006-08-07 23:50:00
4739629,2006,8,20,3.0,2353.0,50.0,2006-08-20 00:03:00,2006-08-19 23:53:00
5314650,2006,9,13,11.0,2359.0,48.0,2006-09-13 00:11:00,2006-09-12 23:59:00
6498563,2006,11,1,2.0,2348.0,46.0,2006-11-01 00:02:00,2006-10-31 23:48:00
6929660,2006,12,3,4.0,2354.0,50.0,2006-12-03 00:04:00,2006-12-02 23:54:00


#### **Issue**: There are values in the `DepTime` and `ArrTime` columns that are greater than 2359.

##### **Define**
- Identify rows where there are values greater than 2359 for each column.
- Convert each value to timestamp and add 1 day.
- Work out the necessary changes to timestamps for various combinations of values greater than 2359.
- Complete any of the changes that are still outstanding.

##### **Code**

In [93]:
# Identify rows where times are not null and are greater than 2359
DepTime_2400_mask = (flights_clean["DepTime"] > 2359) & (flights_clean["DepTime"].notnull())
DepTime_2400_mask.value_counts()

False    7140509
True        1402
Name: DepTime, dtype: int64

In [94]:
flights_clean.loc[DepTime_2400_mask, :][["DepTime"]].describe()

,DepTime
count,1402.000000
mean,2420.412981
std,44.878008
min,2400.000000
25%,2400.000000
50%,2400.000000
75%,2420.000000
max,2930.000000


In [95]:
flights_clean.loc[DepTime_2400_mask, :][["Year", "Month", "DayofMonth", "DepTime", "ActualDepTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ActualDepTime
5266935,2006,9,12,2400.0,NaT
3496551,2006,6,23,2400.0,NaT
5653145,2006,10,17,2443.0,NaT
2027132,2006,4,20,2430.0,NaT
5053161,2006,9,22,2400.0,NaT
3825635,2006,7,9,2400.0,NaT
5570638,2006,10,25,2400.0,NaT
4449764,2006,8,5,2517.0,NaT
6232776,2006,11,2,2400.0,NaT
3821297,2006,7,5,2408.0,NaT


In [96]:
flights_clean.loc[DepTime_2400_mask, "ActualDepTime"] = pd.to_datetime(flights_clean.loc[DepTime_2400_mask, :]["Year"].apply(str) + flights_clean.loc[DepTime_2400_mask, :]["Month"].apply(str).str.zfill(2) + flights_clean.loc[DepTime_2400_mask, :]["DayofMonth"].apply(str).str.zfill(2) + (flights_clean.loc[DepTime_2400_mask, :]["DepTime"] - 2400).apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M") + pd.Timedelta(days = 1)

In [97]:
# Identify rows where times are not null and are not greater than 2359
ArrTime_2400_mask = (flights_clean["ArrTime"] > 2359) & (flights_clean["ArrTime"].notnull())
ArrTime_2400_mask.value_counts()

False    7136002
True        5909
Name: ArrTime, dtype: int64

In [98]:
flights_clean.loc[ArrTime_2400_mask, :][["ArrTime"]].describe()

,ArrTime
count,5909.000000
mean,2433.577763
std,55.143135
min,2400.000000
25%,2400.000000
50%,2410.000000
75%,2440.000000
max,2955.000000


In [99]:
flights_clean.loc[ArrTime_2400_mask, :][["Year", "Month", "DayofMonth", "ArrTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,ArrTime,ActualArrTime
852466,2006,2,28,2516.0,NaT
3220102,2006,6,30,2413.0,NaT
4458903,2006,8,21,2521.0,NaT
2609856,2006,5,7,2439.0,NaT
4560123,2006,8,19,2400.0,NaT
2962470,2006,6,8,2400.0,NaT
4451127,2006,8,9,2410.0,NaT
5666247,2006,10,23,2400.0,NaT
4452110,2006,8,10,2540.0,NaT
2745890,2006,5,1,2400.0,NaT


In [100]:
flights_clean.loc[ArrTime_2400_mask, "ActualArrTime"] = pd.to_datetime(flights_clean.loc[ArrTime_2400_mask, :]["Year"].apply(str) + flights_clean.loc[ArrTime_2400_mask, :]["Month"].apply(str).str.zfill(2) + flights_clean.loc[ArrTime_2400_mask, :]["DayofMonth"].apply(str).str.zfill(2) + (flights_clean.loc[ArrTime_2400_mask, :]["ArrTime"] - 2400).apply(int).apply(str).str.zfill(4), format = "%Y%m%d%H%M") + pd.Timedelta(days = 1)

There are varying combinations of `DepTime` and `ArrTime` where at least one of them is greater than 2359. For each combination, a different approach is required to obtain the corresponding values for the dates in the `ActualDepTime` and `ActualArrTime` timestamp columns.

Where `DepTime` > 2359 and `ArrTime` > 2359:
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day` plus 1 day.
- the date for `ActualArrTime` is the date in `Year`-`Month`-`Day` plus 1 day.

Both of these have already been completed by adding one day during the conversions from *int* to *Timestamp* above.

Where `DepTime` > 2359 and `ArrTime`.isnull():
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day` plus 1 day.
- there isn't a timestamp for `ActualArrTime` as these are diverted flights.

The first one has already been completed by adding one day during the conversions from *int* to *Timestamp* above. Nothing further needs to be done for the second one.

Where `DepTime` < 2400 and `ArrTime` > 2359:
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day`.
- the date for `ActualArrTime` is the date in `Year`-`Month`-`Day` plus 1 day.

The first one is already correct in the original data so nothing further needs to be done. The second one has already been completed by adding one day during the conversions from *int* to *Timestamp* above.

Where `DepTime` > 2359 and `ArrTime` < 2400 and `ActualArrTime` is before midnight:
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day` plus 1 day.
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day`.

The first one has already been completed by adding one day during the conversions from *int* to *Timestamp* above. The second one is already correct in the original data so nothing further needs to be done.

Where `DepTime` > 2359 and `ArrTime` < 2400 and`ActualArrTime` is after midnight:
- the date for `ActualDepTime` is the date in `Year`-`Month`-`Day` plus 1 day.
- the date for `ActualArrTime` is the date in `Year`-`Month`-`Day` plus 1 day.

The first one has already been completed by adding one day during the conversions from *int* to *Timestamp* above. The second one needs to be corrected manually, and I will now do that below.

In [101]:
ActualArrTime_next_day_mask = (flights_clean["DepTime"] > 2359) & ( flights_clean["ArrTime"] < 2400) & \
                              (flights_clean["ArrTime"] < 1800)
ActualArrTime_next_day_mask.value_counts()

False    7141124
True         787
dtype: int64

In [102]:
flights_clean[ActualArrTime_next_day_mask].sample(8)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
5014828,2006,9,30,6,2400.0,2359,59.0,111,US,116,N838AW,59.0,72.0,40.0,-12.0,1.0,LAS,LAX,236,4,15,0,NaN,0,0,0,0,0,0,2006-10-01,2006-09-30 00:59:00,2006-09-30 23:59:00,2006-10-01 01:11:00
4275626,2006,8,24,4,2400.0,2351,153.0,145,YV,2866,N77302,113.0,114.0,88.0,8.0,9.0,LAS,MFR,599,5,20,0,NaN,0,0,0,0,0,0,2006-08-25,2006-08-24 01:53:00,2006-08-24 23:51:00,2006-08-25 01:45:00
2928952,2006,6,12,1,2400.0,2359,112.0,110,US,116,N822AW,72.0,71.0,44.0,2.0,1.0,LAS,LAX,236,6,22,0,NaN,0,0,0,0,0,0,2006-06-13,2006-06-12 01:12:00,2006-06-12 23:59:00,2006-06-13 01:10:00
606964,2006,2,24,5,2400.0,2359,835.0,851,US,925,N904AW,335.0,352.0,315.0,-16.0,1.0,HNL,PHX,2917,3,17,0,NaN,0,0,0,0,0,0,2006-02-25,2006-02-24 08:35:00,2006-02-24 23:59:00,2006-02-25 08:51:00
2331172,2006,5,22,1,2400.0,2359,109.0,108,US,99,N818AW,69.0,69.0,37.0,1.0,1.0,LAS,PHX,256,5,27,0,NaN,0,0,0,0,0,0,2006-05-23,2006-05-22 01:09:00,2006-05-22 23:59:00,2006-05-23 01:08:00
2408940,2006,5,25,4,2400.0,2030,107.0,2140,WN,1269,N610SW,67.0,70.0,49.0,207.0,210.0,CMH,BWI,336,3,15,0,NaN,0,3,0,0,0,204,2006-05-26,2006-05-25 01:07:00,2006-05-25 20:30:00,2006-05-25 21:40:00
3632106,2006,7,1,6,2400.0,2351,220.0,145,YV,6381,N27318,140.0,114.0,86.0,35.0,9.0,LAS,MFR,599,7,47,0,NaN,0,35,0,0,0,0,2006-07-02,2006-07-01 02:20:00,2006-07-01 23:51:00,2006-07-02 01:45:00
5151420,2006,9,11,1,2400.0,10,737.0,734,NW,336,N534US,277.0,264.0,229.0,3.0,-10.0,LAX,DTW,1979,14,34,0,NaN,0,0,0,0,0,0,2006-09-12,2006-09-11 07:37:00,2006-09-11 00:10:00,2006-09-11 07:34:00


In [103]:
flights_clean.loc[ActualArrTime_next_day_mask, "ActualArrTime"] += pd.Timedelta(days = 1)

##### **Test**

In [104]:
flights_clean[(flights_clean["DepTime"] > 2359) & \
              (flights_clean["ArrTime"] > 2359)][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                  "ActualDepTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ArrTime,ActualDepTime,ActualArrTime
5650686,2006,10,19,2535.0,2615.0,2006-10-20 01:35:00,2006-10-20 02:15:00
3207840,2006,6,26,2400.0,2448.0,2006-06-27 00:00:00,2006-06-27 00:48:00
3212045,2006,6,25,2550.0,2647.0,2006-06-26 01:50:00,2006-06-26 02:47:00
3823362,2006,7,14,2600.0,2817.0,2006-07-15 02:00:00,2006-07-15 04:17:00
301820,2006,1,17,2400.0,2407.0,2006-01-18 00:00:00,2006-01-18 00:07:00
3216332,2006,6,22,2400.0,2527.0,2006-06-23 00:00:00,2006-06-23 01:27:00
4456970,2006,8,10,2430.0,2705.0,2006-08-11 00:30:00,2006-08-11 03:05:00
3214009,2006,6,25,2416.0,2622.0,2006-06-26 00:16:00,2006-06-26 02:22:00
5059398,2006,9,18,2429.0,2626.0,2006-09-19 00:29:00,2006-09-19 02:26:00
3221616,2006,6,24,2452.0,2607.0,2006-06-25 00:52:00,2006-06-25 02:07:00


In [105]:
flights_clean[(flights_clean["DepTime"] > 2359) & \
              (flights_clean["ArrTime"].isnull())][["Year", "Month", "DayofMonth", "DepTime", \
                                                    "ArrTime", "ActualDepTime", "ActualArrTime"]]

,Year,Month,DayofMonth,DepTime,ArrTime,ActualDepTime,ActualArrTime
137784,2006,1,22,2400.0,NaN,2006-01-23,NaT
1140741,2006,3,22,2400.0,NaN,2006-03-23,NaT


In [106]:
flights_clean[(flights_clean["DepTime"] < 2400) & \
              (flights_clean["ArrTime"] > 2359)][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                  "ActualDepTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ArrTime,ActualDepTime,ActualArrTime
2617870,2006,5,25,2250.0,2405.0,2006-05-25 22:50:00,2006-05-26 00:05:00
3839453,2006,7,19,2310.0,2524.0,2006-07-19 23:10:00,2006-07-20 01:24:00
2286927,2006,4,16,2252.0,2400.0,2006-04-16 22:52:00,2006-04-17 00:00:00
5665769,2006,10,25,2232.0,2500.0,2006-10-25 22:32:00,2006-10-26 01:00:00
5050481,2006,9,24,2157.0,2440.0,2006-09-24 21:57:00,2006-09-25 00:40:00
5664310,2006,10,23,2203.0,2419.0,2006-10-23 22:03:00,2006-10-24 00:19:00
5063134,2006,9,12,2236.0,2402.0,2006-09-12 22:36:00,2006-09-13 00:02:00
3837214,2006,7,10,2150.0,2423.0,2006-07-10 21:50:00,2006-07-11 00:23:00
5059697,2006,9,14,2349.0,2433.0,2006-09-14 23:49:00,2006-09-15 00:33:00
4461231,2006,8,1,2201.0,2413.0,2006-08-01 22:01:00,2006-08-02 00:13:00


In [107]:
flights_clean[(flights_clean["DepTime"] > 2359) & ( flights_clean["ArrTime"] < 2400) & \
              (flights_clean["ArrTime"] >= 1800)][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                   "ActualDepTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ArrTime,ActualDepTime,ActualArrTime
1419507,2006,3,9,2400.0,2346.0,2006-03-10 00:00:00,2006-03-09 23:46:00
3831684,2006,7,4,2400.0,2345.0,2006-07-05 00:00:00,2006-07-04 23:45:00
4450528,2006,8,13,2400.0,2347.0,2006-08-14 00:00:00,2006-08-13 23:47:00
3817918,2006,7,28,2418.0,2355.0,2006-07-29 00:18:00,2006-07-28 23:55:00
306135,2006,1,3,2402.0,2352.0,2006-01-04 00:02:00,2006-01-03 23:52:00
4456165,2006,8,31,2404.0,2359.0,2006-09-01 00:04:00,2006-08-31 23:59:00
1419505,2006,3,7,2412.0,2346.0,2006-03-08 00:12:00,2006-03-07 23:46:00
3494498,2006,6,22,2400.0,2351.0,2006-06-23 00:00:00,2006-06-22 23:51:00
4442442,2006,8,29,2400.0,2337.0,2006-08-30 00:00:00,2006-08-29 23:37:00
2606931,2006,5,14,2400.0,2341.0,2006-05-15 00:00:00,2006-05-14 23:41:00


In [108]:
flights_clean[(flights_clean["DepTime"] > 2359) & ( flights_clean["ArrTime"] < 2400) & \
              (flights_clean["ArrTime"] < 1800)][["Year", "Month", "DayofMonth", "DepTime", "ArrTime", \
                                                  "ActualDepTime", "ActualArrTime"]].sample(10)

,Year,Month,DayofMonth,DepTime,ArrTime,ActualDepTime,ActualArrTime
708740,2006,2,22,2400.0,215.0,2006-02-23,2006-02-23 02:15:00
2557411,2006,5,11,2400.0,127.0,2006-05-12,2006-05-12 01:27:00
2450431,2006,5,28,2400.0,115.0,2006-05-29,2006-05-29 01:15:00
3799212,2006,7,5,2400.0,752.0,2006-07-06,2006-07-06 07:52:00
5611132,2006,10,22,2400.0,521.0,2006-10-23,2006-10-23 05:21:00
1049715,2006,2,23,2400.0,435.0,2006-02-24,2006-02-24 04:35:00
657770,2006,2,17,2400.0,108.0,2006-02-18,2006-02-18 01:08:00
6232784,2006,11,16,2400.0,652.0,2006-11-17,2006-11-17 06:52:00
4279191,2006,8,28,2400.0,300.0,2006-08-29,2006-08-29 03:00:00
3761968,2006,7,9,2400.0,41.0,2006-07-10,2006-07-10 00:41:00


#### **Issue**: There are negative values in the `AirTime` column.

##### **Define**
- Identify rows where negative values are present.
- Drop identified rows.
- Reset the index of the dataframe.

##### **Code**

In [109]:
AirTime_negative_mask = flights_clean["AirTime"] < 0
AirTime_negative_mask.value_counts()

False    7138651
True        3260
Name: AirTime, dtype: int64

In [110]:
flights_clean[AirTime_negative_mask].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,3260.0,3260.000000,3260.000000,3260.000000,3260.000000,3260.000000,3260.000000,3260.000000,3260,3260.000000,3260,3260.000000,3260.000000,3260.000000,3260.000000,3260.000000,3260,3260,3260.000000,3260.000000,3260.000000,3260.0,0,3260.0,3260.000000,3260.000000,3260.000000,3260.000000,3260.000000,3260,3260,3260,3260
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,159,NaN,NaN,NaN,NaN,NaN,81,116,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,2945,2978,2687,3060
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EV,NaN,N926EV,NaN,NaN,NaN,NaN,NaN,ATL,ATL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-25 23:30:00,2006-09-19 00:20:00,2006-03-09 22:35:00,2006-04-04 00:08:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3260,NaN,36,NaN,NaN,NaN,NaN,NaN,2946,215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,4,3
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-01 21:50:00,2006-01-02 00:04:00,2006-01-01 20:07:00,2006-01-01 20:50:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-10-31 23:54:00,2006-11-01 02:05:00,2006-10-31 20:23:00,2006-10-31 22:30:00
mean,2006.0,5.791718,15.534969,3.938344,2299.000613,2140.146012,2460.078834,1694.261656,NaN,4478.741411,NaN,106.409509,107.520552,-1359.551840,91.890798,93.001840,NaN,NaN,574.715031,1444.873006,21.088344,0.0,NaN,0.0,31.577607,36.519325,14.290798,0.230061,9.204601,NaN,NaN,NaN,NaN
std,0.0,2.741361,8.788452,2.019285,93.397043,164.222630,62.664746,980.737952,NaN,225.532379,NaN,35.930819,31.861986,30.941592,97.505646,99.249176,NaN,NaN,256.247688,3.846687,15.058366,0.0,NaN,0.0,84.583603,64.594127,43.620987,4.948754,35.517144,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,1908.000000,155.000000,2402.000000,1.000000,NaN,4084.000000,NaN,23.000000,38.000000,-1425.000000,-32.000000,-63.000000,NaN,NaN,79.000000,1441.000000,0.000000,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,3.750000,8.000000,2.000000,2240.000000,2105.000000,2418.000000,51.000000,NaN,4289.000000,NaN,80.000000,78.000000,-1385.000000,24.000000,23.000000,NaN,NaN,357.000000,1443.000000,12.000000,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


In the above table, it can be observed that there are several commonalities for these entries where `ArrTime` is negative:
- All the values for `ArrTime` are greater than 2400. If this column was used in computing some of the other columns in the dataframe when this dataset was being compiled, it is a possible source of the negative values in the `ArrTime` column.
- All the values in the `TaxiIn` column are outliers, greater than 1441 minutes, which is over 24 hours and is unlikely to be correct.
- All the flights are by the same carrier, EV. It is highly unlikely that this is coincidental. It is possible there is a systematic error in this specific carrier's representation or computation of some of the times and/or time intervals in the dataframe, that leads to negative values of `AirTime`.

For most other entries in the table, the actual elapsed time is the sum of the air time, taxi in time, and taxi out time. Having any 3 of these values would make it possible to compute the 4th. This is not possible for the `AirTime` values in these entries since the values for `TaxiIn` are also evidently incorrect. Therefore, I will drop all the rows.

In [111]:
flights_clean.drop(flights_clean[AirTime_negative_mask].index, axis = 0, inplace = True)

In [112]:
flights_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [113]:
(flights_clean["AirTime"] < 0).value_counts()

False    7138651
Name: AirTime, dtype: int64

#### **Issue**: There are negative values in the CRSElapsedTime column.

##### **Define**

##### **Code**

In [114]:
CRSElapsed_negative_mask = flights_clean["CRSElapsedTime"] < 0
CRSElapsed_negative_mask.value_counts()

False    7138624
True          27
Name: CRSElapsedTime, dtype: int64

In [115]:
flights_clean[CRSElapsed_negative_mask]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
381963,2006,1,2,1,923.0,924,NaN,1000,MQ,4469,N931AE,NaN,-24.0,NaN,NaN,-1.0,ORD,TVC,224,0,22,0,NaN,1,0,0,0,0,0,2006-01-02 09:23:00,NaT,2006-01-02 09:24:00,2006-01-02 10:00:00
927298,2006,2,5,7,752.0,751,NaN,748,MQ,4331,N677AE,NaN,-3.0,NaN,NaN,1.0,MQT,TVC,142,0,16,0,NaN,1,0,0,0,0,0,2006-02-05 07:52:00,NaT,2006-02-05 07:51:00,2006-02-05 07:48:00
927308,2006,2,10,5,1220.0,751,NaN,748,MQ,4331,N629AE,NaN,-3.0,NaN,NaN,269.0,MQT,TVC,142,0,22,0,NaN,1,0,0,0,0,0,2006-02-10 12:20:00,NaT,2006-02-10 07:51:00,2006-02-10 07:48:00
930727,2006,2,16,4,1206.0,1054,NaN,1114,MQ,4469,N663AR,NaN,-40.0,NaN,NaN,72.0,ORD,TVC,224,0,11,0,NaN,1,0,0,0,0,0,2006-02-16 12:06:00,NaT,2006-02-16 10:54:00,2006-02-16 11:14:00
2684771,2006,5,10,3,1812.0,1810,NaN,1846,MQ,3990,N923AE,NaN,-24.0,NaN,NaN,2.0,ORD,MQT,303,0,16,0,NaN,1,0,0,0,0,0,2006-05-10 18:12:00,NaT,2006-05-10 18:10:00,2006-05-10 18:46:00
2694952,2006,5,15,1,947.0,800,NaN,741,MQ,4331,N690AE,NaN,-19.0,NaN,NaN,107.0,MQT,TVC,142,0,3,0,NaN,1,0,0,0,0,0,2006-05-15 09:47:00,NaT,2006-05-15 08:00:00,2006-05-15 07:41:00
3042438,2006,6,17,6,NaN,1950,NaN,1945,YV,7225,N650ML,NaN,-5.0,NaN,NaN,NaN,ORD,ATW,160,0,0,1,C,0,0,0,0,0,0,NaT,NaT,2006-06-17 19:50:00,2006-06-17 19:45:00
4521905,2006,8,1,2,1822.0,1810,NaN,1846,MQ,3990,N623AE,NaN,-24.0,NaN,NaN,12.0,ORD,MQT,303,0,25,0,NaN,1,0,0,0,0,0,2006-08-01 18:22:00,NaT,2006-08-01 18:10:00,2006-08-01 18:46:00
4521929,2006,8,29,2,1856.0,1810,NaN,1846,MQ,3990,N616AE,NaN,-24.0,NaN,NaN,46.0,ORD,MQT,303,0,21,0,NaN,1,0,0,0,0,0,2006-08-29 18:56:00,NaT,2006-08-29 18:10:00,2006-08-29 18:46:00
4605452,2006,8,9,3,1632.0,1612,NaN,1701,AA,550,N5TJAA,NaN,-71.0,NaN,NaN,20.0,LAS,DFW,1055,0,30,0,NaN,1,0,0,0,0,0,2006-08-09 16:32:00,NaT,2006-08-09 16:12:00,2006-08-09 17:01:00


In [116]:
CRSElapsed_drop_mask = CRSElapsed_negative_mask & (flights_clean["Diverted"] == 0) & (flights_clean["Cancelled"] == 0)
CRSElapsed_drop_mask.value_counts()

False    7138650
True           1
dtype: int64

In [117]:
flights_clean.drop(flights_clean[CRSElapsed_drop_mask].index, axis = 0, inplace = True)

In [118]:
flights_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [119]:
(flights_clean["CRSElapsedTime"] < 0).value_counts()

False    7138624
True          26
Name: CRSElapsedTime, dtype: int64

#### **Issue**: There are outliers in the `ActualElapsedTime`, `CRSElapsedTime`, `ArrDelay`, `DepDelay`, `TaxiIn`, `TaxiOut` and `Distance` columns.

##### **Define**

##### **Code**

I will look at the values for each column at an extended range of percentiles.

In [120]:
flights_clean[["ActualElapsedTime", "CRSElapsedTime", "ArrDelay", "DepDelay", "TaxiIn", \
               "TaxiOut", "Distance"]].describe(percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,TaxiIn,TaxiOut,Distance
count,7.000535e+06,7.138650e+06,7.000535e+06,7.016720e+06,7.138650e+06,7.138650e+06,7.138650e+06
mean,1.260693e+02,1.271935e+02,8.644040e+00,1.005508e+01,6.405026e+00,1.574287e+01,7.280680e+02
std,7.133073e+01,7.032690e+01,3.648017e+01,3.339279e+01,4.959764e+00,1.126857e+01,5.746267e+02
min,5.000000e+00,-9.700000e+01,-5.920000e+02,-1.200000e+03,0.000000e+00,0.000000e+00,1.100000e+01
5%,4.900000e+01,5.200000e+01,-2.000000e+01,-9.000000e+00,2.000000e+00,6.000000e+00,1.420000e+02
10%,5.700000e+01,6.000000e+01,-1.600000e+01,-7.000000e+00,3.000000e+00,7.000000e+00,1.970000e+02
25%,7.500000e+01,7.600000e+01,-9.000000e+00,-4.000000e+00,4.000000e+00,1.000000e+01,3.160000e+02
50%,1.070000e+02,1.090000e+02,-1.000000e+00,0.000000e+00,5.000000e+00,1.300000e+01,5.740000e+02
75%,1.560000e+02,1.570000e+02,1.300000e+01,9.000000e+00,8.000000e+00,1.800000e+01,9.560000e+02
90%,2.250000e+02,2.250000e+02,4.100000e+01,3.700000e+01,1.100000e+01,2.600000e+01,1.541000e+03


I have already investigated the negative values in the `CRSElapsedTime` column in the  previous subsection. I will now investigate the outliers that are at the other extreme.

In [121]:
(flights_clean["CRSElapsedTime"] > 660).value_counts()

False    7138648
True           2
Name: CRSElapsedTime, dtype: int64

In [122]:
flights_clean[(flights_clean["CRSElapsedTime"] > 660)]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
2935676,2006,6,27,2,2059.0,850,NaN,2301,US,796,N648AW,NaN,1031.0,NaN,NaN,729.0,BWI,LAS,2106,0,22,0,NaN,1,0,0,0,0,0,2006-06-27 20:59:00,NaT,2006-06-27 08:50:00,2006-06-27 23:01:00
5597709,2006,10,12,4,920.0,912,NaN,408,US,578,N637AW,NaN,956.0,NaN,NaN,8.0,PHX,TPA,1788,0,24,0,NaN,1,0,0,0,0,0,2006-10-12 09:20:00,NaT,2006-10-12 09:12:00,2006-10-12 04:08:00


Both rows show flights that were diverted, one of which had a significantly delayed departure. I will not be removing these rows, as I will analyze diverted flights.

I move on to assessing the `ActualElapsedTime` column.

In [123]:
(flights_clean["ActualElapsedTime"] > 900).value_counts()

False    7138590
True          60
Name: ActualElapsedTime, dtype: int64

In [124]:
flights_clean[(flights_clean["ActualElapsedTime"] > 900)]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
503699,2006,1,2,1,2036.0,1945,1205.0,2315,B6,47,0,929.0,210.0,877.0,770.0,51.0,JFK,FLL,1069,3,49,0,NaN,0,0,0,726,0,44,2006-01-02 20:36:00,2006-01-02 12:05:00,2006-01-02 19:45:00,2006-01-02 23:15:00
504031,2006,1,3,2,225.0,2120,513.0,2359,B6,50,N519JB,1608.0,159.0,1587.0,314.0,-1135.0,FLL,JFK,1069,8,13,0,NaN,0,5,0,9,0,300,2006-01-03 02:25:00,2006-01-03 05:13:00,2006-01-03 21:20:00,2006-01-03 23:59:00
504066,2006,1,3,2,643.0,2245,754.0,2355,B6,128,N507JB,1511.0,70.0,1487.0,479.0,-962.0,JFK,BTV,267,3,21,0,NaN,0,0,478,1,0,0,2006-01-03 06:43:00,2006-01-03 07:54:00,2006-01-03 22:45:00,2006-01-03 23:55:00
507466,2006,1,13,5,506.0,1955,622.0,2105,B6,1019,N187JB,1516.0,70.0,1493.0,557.0,-889.0,BOS,JFK,187,12,11,0,NaN,0,14,0,6,0,537,2006-01-13 05:06:00,2006-01-13 06:22:00,2006-01-13 19:55:00,2006-01-13 21:05:00
507547,2006,1,13,5,43.0,2010,314.0,2305,B6,382,N508JB,1591.0,175.0,1570.0,249.0,-1167.0,FLL,LGA,1076,5,16,0,NaN,0,0,0,107,0,142,2006-01-13 00:43:00,2006-01-13 03:14:00,2006-01-13 20:10:00,2006-01-13 23:05:00
513525,2006,1,30,1,8.0,1850,124.0,2000,B6,1033,N193JB,1516.0,70.0,1495.0,324.0,-1122.0,BTV,JFK,267,9,12,0,NaN,0,14,304,6,0,0,2006-01-30 00:08:00,2006-01-30 01:24:00,2006-01-30 18:50:00,2006-01-30 20:00:00
513963,2006,1,31,2,148.0,2040,501.0,2355,B6,1069,N184JB,1693.0,255.0,1611.0,306.0,-1132.0,JFK,AUS,1522,6,16,0,NaN,0,290,0,16,0,0,2006-01-31 01:48:00,2006-01-31 05:01:00,2006-01-31 20:40:00,2006-01-31 23:55:00
1041450,2006,2,3,5,324.0,2040,654.0,2359,B6,455,N523JB,1650.0,199.0,1634.0,415.0,-1036.0,BOS,FLL,1237,4,12,0,NaN,0,14,0,11,0,390,2006-02-03 03:24:00,2006-02-03 06:54:00,2006-02-03 20:40:00,2006-02-03 23:59:00
1041487,2006,2,3,5,158.0,1540,534.0,1900,B6,447,N550JB,1656.0,200.0,1643.0,634.0,-822.0,BOS,TPA,1185,4,9,0,NaN,0,618,0,16,0,0,2006-02-03 01:58:00,2006-02-03 05:34:00,2006-02-03 15:40:00,2006-02-03 19:00:00
1041623,2006,2,3,5,15.0,1905,320.0,2205,B6,43,N520JB,1625.0,180.0,1599.0,315.0,-1130.0,JFK,MCO,944,7,19,0,NaN,0,0,0,5,0,310,2006-02-03 00:15:00,2006-02-03 03:20:00,2006-02-03 19:05:00,2006-02-03 22:05:00


The departure delay values are incorrect, likely due to the original calculations when this dataset was first compiled not taking into account the fact that the delays overlapped into the next day. There are few enough that I will drop them.

I will now assess the `ArrDelay` column. I will start with values of `ArrDelay` that are very large and negative.

In [125]:
(flights_clean["ArrDelay"] < -120).value_counts()

False    7138630
True          20
Name: ArrDelay, dtype: int64

In [126]:
flights_clean[(flights_clean["ArrDelay"] < -120)]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
142161,2006,1,28,6,645.0,648,716.0,925,YV,7396,N503MJ,31.0,157.0,15.0,-129.0,-3.0,AUS,ORD,978,7,9,0,NaN,0,0,0,0,0,0,2006-01-28 06:45:00,2006-01-28 07:16:00,2006-01-28 06:48:00,2006-01-28 09:25:00
295660,2006,1,4,3,930.0,1200,1113.0,1330,EV,4143,N842AS,103.0,90.0,66.0,-137.0,-150.0,CVG,ROC,461,10,27,0,NaN,0,0,0,0,0,0,2006-01-04 09:30:00,2006-01-04 11:13:00,2006-01-04 12:00:00,2006-01-04 13:30:00
495184,2006,1,12,4,1441.0,1803,1535.0,1852,AS,139,N747AS,54.0,49.0,44.0,-197.0,-202.0,AKN,ANC,289,4,6,0,NaN,0,0,0,0,0,0,2006-01-12 14:41:00,2006-01-12 15:35:00,2006-01-12 18:03:00,2006-01-12 18:52:00
1468543,2006,3,3,5,1917.0,520,2035.0,557,HA,106,N487HA,78.0,37.0,64.0,-562.0,-603.0,HNL,OGG,100,5,9,0,NaN,0,0,0,0,0,0,2006-03-03 19:17:00,2006-03-03 20:35:00,2006-03-04 05:20:00,2006-03-04 05:57:00
1468755,2006,3,10,5,2023.0,625,2151.0,702,HA,114,N480HA,88.0,37.0,62.0,-551.0,-602.0,HNL,OGG,100,5,21,0,NaN,0,0,0,0,0,0,2006-03-10 20:23:00,2006-03-10 21:51:00,2006-03-11 06:25:00,2006-03-11 07:02:00
1469365,2006,3,31,5,203.0,1205,325.0,1234,HA,144,N479HA,82.0,29.0,70.0,-549.0,-602.0,LIH,HNL,102,6,6,0,NaN,0,0,0,0,0,0,2006-03-31 02:03:00,2006-03-31 03:25:00,2006-03-31 12:05:00,2006-03-31 12:34:00
1470368,2006,3,21,2,541.0,1545,725.0,1636,HA,262,N486HA,104.0,51.0,88.0,-551.0,-604.0,HNL,ITO,216,6,10,0,NaN,0,0,0,0,0,0,2006-03-21 05:41:00,2006-03-21 07:25:00,2006-03-21 15:45:00,2006-03-21 16:36:00
1470664,2006,3,17,5,445.0,1435,544.0,1512,HA,320,N475HA,59.0,37.0,49.0,-568.0,-590.0,HNL,OGG,100,5,5,0,NaN,0,0,0,0,0,0,2006-03-17 04:45:00,2006-03-17 05:44:00,2006-03-17 14:35:00,2006-03-17 15:12:00
1471158,2006,3,20,1,141.0,1140,303.0,1227,HA,383,N486HA,82.0,47.0,69.0,-564.0,-599.0,ITO,HNL,216,5,8,0,NaN,0,0,0,0,0,0,2006-03-20 01:41:00,2006-03-20 03:03:00,2006-03-20 11:40:00,2006-03-20 12:27:00
1471623,2006,3,20,1,629.0,1635,737.0,1712,HA,546,N481HA,68.0,37.0,55.0,-575.0,-606.0,HNL,OGG,100,4,9,0,NaN,0,0,0,0,0,0,2006-03-20 06:29:00,2006-03-20 07:37:00,2006-03-20 16:35:00,2006-03-20 17:12:00


All the values seem to be accurate. None of these rows need to be cleaned.

I will now look at values of `ArrDelay` that are very large and positive.

In [127]:
(flights_clean["ArrDelay"] > 1200).value_counts()

False    7138600
True          50
Name: ArrDelay, dtype: int64

In [128]:
flights_clean[flights_clean["ArrDelay"] > 1200].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
2755338,2006,5,6,6,1019.0,1325,1852.0,2144,AA,162,N393AA,333.0,319.0,313.0,1268.0,1254.0,HNL,LAX,2556,6,14,0,NaN,0,1254,0,14,0,0,2006-05-06 10:19:00,2006-05-06 18:52:00,2006-05-06 13:25:00,2006-05-06 21:44:00
6943060,2006,12,12,2,12.0,2350,719.0,720,NW,218,N587NW,307.0,330.0,268.0,1439.0,1462.0,HNL,PDX,2603,6,33,0,NaN,0,1439,0,0,0,0,2006-12-12 00:12:00,2006-12-12 07:19:00,2006-12-12 23:50:00,2006-12-13 07:20:00
6747561,2006,12,23,6,610.0,800,1317.0,1503,UA,72,N664UA,307.0,303.0,275.0,1334.0,1330.0,HNL,SFO,2398,19,13,0,NaN,0,367,0,4,0,963,2006-12-23 06:10:00,2006-12-23 13:17:00,2006-12-23 08:00:00,2006-12-23 15:03:00
7033093,2006,12,19,2,1550.0,1835,1842.0,2130,AA,2009,N424AA,232.0,235.0,188.0,1272.0,1275.0,PHL,DFW,1302,27,17,0,NaN,0,1272,0,0,0,0,2006-12-19 15:50:00,2006-12-19 18:42:00,2006-12-19 18:35:00,2006-12-19 21:30:00
3369438,2006,6,25,7,1339.0,1859,1728.0,2108,AA,817,N447AA,289.0,189.0,163.0,1220.0,1120.0,DCA,DFW,1192,41,85,0,NaN,0,642,0,100,0,478,2006-06-25 13:39:00,2006-06-25 17:28:00,2006-06-25 18:59:00,2006-06-25 21:08:00
5224989,2006,9,24,7,554.0,800,1118.0,1325,AA,1612,N484AA,204.0,205.0,181.0,1313.0,1314.0,OAK,DFW,1456,12,11,0,NaN,0,1313,0,0,0,0,2006-09-24 05:54:00,2006-09-24 11:18:00,2006-09-24 08:00:00,2006-09-24 13:25:00
3925427,2006,7,11,2,603.0,905,1257.0,1607,NW,258,N512US,234.0,242.0,205.0,1250.0,1258.0,PHX,DTW,1671,11,18,0,NaN,0,1228,0,0,0,22,2006-07-11 06:03:00,2006-07-11 12:57:00,2006-07-11 09:05:00,2006-07-11 16:07:00
7033397,2006,12,3,7,1115.0,1410,1551.0,1850,AA,2022,N5TXAA,156.0,160.0,128.0,1261.0,1265.0,LAS,DFW,1055,14,14,0,NaN,0,1261,0,0,0,0,2006-12-03 11:15:00,2006-12-03 15:51:00,2006-12-03 14:10:00,2006-12-03 18:50:00
1528514,2006,3,17,5,726.0,1029,915.0,1230,NW,539,N601NW,109.0,121.0,79.0,1245.0,1257.0,LGA,DTW,501,7,23,0,NaN,0,1245,0,0,0,0,2006-03-17 07:26:00,2006-03-17 09:15:00,2006-03-17 10:29:00,2006-03-17 12:30:00
3723228,2006,7,2,7,1209.0,1310,2005.0,2126,UA,56,N545UA,296.0,316.0,270.0,1359.0,1379.0,KOA,LAX,2504,12,14,0,NaN,0,1086,0,0,0,273,2006-07-02 12:09:00,2006-07-02 20:05:00,2006-07-02 13:10:00,2006-07-02 21:26:00


In [129]:
flights_clean[flights_clean["ArrDelay"] > 1200].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,50.0,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50,50.000000,50,50.00000,50.000000,50.000000,50.000000,50.000000,50,50,50.000000,50.000000,50.000000,50.0,0,50.0,50.000000,50.000000,50.000000,50.0,50.000000,50,50,50,50
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,49,NaN,NaN,NaN,NaN,NaN,37,14,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,50,50,50,50
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NW,NaN,N484AA,NaN,NaN,NaN,NaN,NaN,LAS,MSP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-05 08:35:00,2006-01-05 13:24:00,2006-01-05 10:10:00,2006-01-05 15:15:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,NaN,2,NaN,NaN,NaN,NaN,NaN,4,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-05 08:35:00,2006-01-05 13:24:00,2006-01-05 10:10:00,2006-01-05 15:15:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-31 06:24:00,2006-12-31 12:14:00,2006-12-31 07:45:00,2006-12-31 13:35:00
mean,2006.0,7.420000,17.000000,4.380000,1066.600000,1301.700000,1409.480000,1597.060000,NaN,1327.860000,NaN,246.88000,189.760000,214.280000,1329.220000,1272.100000,NaN,NaN,1292.220000,13.540000,19.060000,0.0,NaN,0.0,1101.160000,0.020000,117.200000,0.0,110.840000,NaN,NaN,NaN,NaN
std,0.0,3.476393,8.607069,2.009061,445.259887,419.335298,418.764375,409.624238,NaN,1590.442808,NaN,258.23527,103.426511,257.700114,116.946576,290.669056,NaN,NaN,920.090242,25.368476,12.171244,0.0,NaN,0.0,460.798081,0.141421,365.722712,0.0,303.039053,NaN,NaN,NaN,NaN
min,2006.0,1.000000,2.000000,1.000000,12.000000,613.000000,522.000000,558.000000,NaN,1.000000,NaN,38.00000,45.000000,20.000000,1201.000000,-38.000000,NaN,NaN,21.000000,2.000000,7.000000,0.0,NaN,0.0,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,4.000000,11.000000,3.000000,728.500000,1005.500000,1138.000000,1352.750000,NaN,277.000000,NaN,131.50000,123.000000,113.750000,1251.250000,1252.500000,NaN,NaN,572.750000,6.000000,13.000000,0.0,NaN,0.0,1204.750000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN


The entries all have large values for at least one type of delay, most frequently carrier delays. Therefore, the values are valid and need not be cleaned. A few entries overlap with those that have a large value for `ActualElapsedTime`, which have already been investigated above, and will be dropped later below.

I will now move on to the `DepDelay` column, starting with values that are negative.

In [130]:
(flights_clean["DepDelay"] < -150).value_counts()

False    7138572
True          78
Name: DepDelay, dtype: int64

In [131]:
flights_clean[flights_clean["DepDelay"] < -150].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
4051347,2006,7,22,6,11.0,2005,218.0,2155,B6,1080,N599JB,1567.0,110.0,1516.0,263.0,-1194.0,CLT,JFK,541,7,44,0,NaN,0,0,0,36,0,227,2006-07-22 00:11:00,2006-07-22 02:18:00,2006-07-22 20:05:00,2006-07-22 21:55:00
3435228,2006,6,27,2,217.0,2210,1020.0,605,B6,420,N621JB,1743.0,295.0,1902.0,255.0,-1193.0,PHX,BOS,2300,4,17,0,NaN,0,0,0,8,0,247,2006-06-27 02:17:00,2006-06-27 10:20:00,2006-06-27 22:10:00,2006-06-28 06:05:00
4050887,2006,7,21,5,102.0,2050,923.0,500,B6,358,N522JB,1761.0,310.0,1914.0,263.0,-1188.0,BUR,JFK,2465,15,12,0,NaN,0,12,0,11,0,240,2006-07-21 01:02:00,2006-07-21 09:23:00,2006-07-21 20:50:00,2006-07-22 05:00:00
1468543,2006,3,3,5,1917.0,520,2035.0,557,HA,106,N487HA,78.0,37.0,64.0,-562.0,-603.0,HNL,OGG,100,5,9,0,NaN,0,0,0,0,0,0,2006-03-03 19:17:00,2006-03-03 20:35:00,2006-03-04 05:20:00,2006-03-04 05:57:00
3424584,2006,6,1,4,33.0,2030,405.0,2355,B6,171,N597JB,1832.0,385.0,1590.0,250.0,-1197.0,JFK,SMF,2521,4,58,0,NaN,0,0,60,7,0,183,2006-06-01 00:33:00,2006-06-01 04:05:00,2006-06-01 20:30:00,2006-06-01 23:55:00
2064148,2006,4,4,2,246.0,1250,419.0,1333,HA,148,N478HA,93.0,43.0,79.0,-554.0,-604.0,HNL,KOA,163,5,9,0,NaN,0,0,0,0,0,0,2006-04-04 02:46:00,2006-04-04 04:19:00,2006-04-04 12:50:00,2006-04-04 13:33:00
1441037,2006,3,15,3,908.0,1908,2044.0,2046,EV,4896,N970EV,696.0,98.0,86.0,-2.0,-600.0,ILM,ATL,377,8,602,0,NaN,0,0,0,0,0,0,2006-03-15 09:08:00,2006-03-15 20:44:00,2006-03-15 19:08:00,2006-03-15 20:46:00
1470664,2006,3,17,5,445.0,1435,544.0,1512,HA,320,N475HA,59.0,37.0,49.0,-568.0,-590.0,HNL,OGG,100,5,5,0,NaN,0,0,0,0,0,0,2006-03-17 04:45:00,2006-03-17 05:44:00,2006-03-17 14:35:00,2006-03-17 15:12:00
3427110,2006,6,7,3,303.0,2230,1122.0,645,B6,498,N583JB,1759.0,315.0,1922.0,277.0,-1167.0,SEA,BOS,2496,4,13,0,NaN,0,3,0,4,0,270,2006-06-07 03:03:00,2006-06-07 11:22:00,2006-06-07 22:30:00,2006-06-08 06:45:00
2064026,2006,4,28,5,118.0,1125,656.0,1154,HA,140,N481HA,338.0,29.0,328.0,-298.0,-607.0,LIH,HNL,102,3,7,0,NaN,0,0,0,0,0,0,2006-04-28 01:18:00,2006-04-28 06:56:00,2006-04-28 11:25:00,2006-04-28 11:54:00


In [132]:
flights_clean[flights_clean["DepDelay"] < -150].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,78.0,78.000000,78.000000,78.000000,78.000000,78.000000,74.000000,78.000000,78,78.000000,78,74.000000,78.000000,74.000000,74.000000,78.000000,78,78,78.000000,78.000000,78.000000,78.0,0,78.000000,78.000000,78.000000,78.000000,78.0,78.000000,78,74,78,78
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,60,NaN,NaN,NaN,NaN,NaN,27,28,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,77,74,73,76
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B6,NaN,N481HA,NaN,NaN,NaN,NaN,NaN,JFK,JFK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-07-27 00:34:00,2006-01-08 16:30:00,2006-06-01 20:30:00,2006-06-01 23:55:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59,NaN,3,NaN,NaN,NaN,NaN,NaN,18,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,3,2
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-03 02:25:00,2006-01-03 05:13:00,2006-01-03 21:20:00,2006-01-03 23:55:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-31 12:51:00,2006-12-31 13:45:00,2006-12-31 18:07:00,2006-12-31 19:02:00
mean,2006.0,4.717949,15.128205,4.371795,423.166667,1855.858974,737.270270,1675.653846,NaN,506.935897,NaN,1257.756757,159.282051,1214.432432,176.040541,-932.179487,NaN,NaN,1004.615385,6.653846,38.217949,0.0,NaN,0.051282,45.602564,48.294872,53.923077,0.0,120.192308,NaN,NaN,NaN,NaN
std,0.0,2.373542,8.754011,1.588179,528.418135,421.052723,544.148129,716.670890,NaN,899.303169,NaN,675.134098,109.109250,684.212337,383.509013,284.643534,NaN,NaN,871.307097,4.826407,73.477560,0.0,NaN,0.222000,135.302292,135.221330,131.673422,0.0,155.783154,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,1.000000,520.000000,124.000000,15.000000,NaN,1.000000,NaN,52.000000,29.000000,17.000000,-592.000000,-1200.000000,NaN,NaN,79.000000,0.000000,5.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,2.250000,8.500000,3.000000,53.750000,1737.000000,368.500000,1172.250000,NaN,106.250000,NaN,897.500000,70.000000,615.750000,67.500000,-1161.000000,NaN,NaN,264.750000,4.000000,12.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN


Most of the entries have very large values in the `ActualElapsedTime` column. Those that do not seem to have values that are large and negative but correct. A few of them are also diverted flights. No cleaning is necessary for these.

In [133]:
(flights_clean["DepDelay"] > 720).value_counts()

False    7138163
True         487
Name: DepDelay, dtype: int64

In [134]:
flights_clean[flights_clean["DepDelay"] > 720].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
1401024,2006,3,11,6,1244.0,1940,1440.0,2132,UA,1519,N438UA,176.0,172.0,142.0,1028.0,1024.0,MCO,ORD,1005,6,28,0,NaN,0,359,0,4,0,665,2006-03-11 12:44:00,2006-03-11 14:40:00,2006-03-11 19:40:00,2006-03-11 21:32:00
4551626,2006,8,1,2,632.0,1539,821.0,1737,NW,341,N515US,289.0,298.0,269.0,884.0,893.0,DTW,SFO,2079,4,16,0,NaN,0,884,0,0,0,0,2006-08-01 06:32:00,2006-08-01 08:21:00,2006-08-01 15:39:00,2006-08-01 17:37:00
5797835,2006,10,29,7,1515.0,2310,2223.0,615,AA,286,N329AA,308.0,305.0,285.0,968.0,965.0,LIH,LAX,2615,8,15,0,NaN,0,0,0,7,0,961,2006-10-29 15:15:00,2006-10-29 22:23:00,2006-10-29 23:10:00,2006-10-30 06:15:00
1580771,2006,3,2,4,659.0,1104,952.0,1402,AA,804,N547AA,173.0,178.0,151.0,1190.0,1195.0,FLL,LGA,1076,11,11,0,NaN,0,1190,0,0,0,0,2006-03-02 06:59:00,2006-03-02 09:52:00,2006-03-02 11:04:00,2006-03-02 14:02:00
1598268,2006,3,9,4,731.0,1720,1049.0,2027,AA,1468,N4YDAA,138.0,127.0,104.0,862.0,851.0,BNA,LGA,764,25,9,0,NaN,0,686,0,11,0,165,2006-03-09 07:31:00,2006-03-09 10:49:00,2006-03-09 17:20:00,2006-03-09 20:27:00
5144943,2006,9,15,5,611.0,1230,1325.0,1944,NW,212,N378NW,254.0,254.0,233.0,1061.0,1061.0,SEA,DTW,1927,7,14,0,NaN,0,1061,0,0,0,0,2006-09-15 06:11:00,2006-09-15 13:25:00,2006-09-15 12:30:00,2006-09-15 19:44:00
2755338,2006,5,6,6,1019.0,1325,1852.0,2144,AA,162,N393AA,333.0,319.0,313.0,1268.0,1254.0,HNL,LAX,2556,6,14,0,NaN,0,1254,0,14,0,0,2006-05-06 10:19:00,2006-05-06 18:52:00,2006-05-06 13:25:00,2006-05-06 21:44:00
5238872,2006,9,22,5,656.0,1450,1150.0,2005,AA,2400,N446AA,174.0,195.0,159.0,945.0,966.0,PHX,ORD,1440,5,10,0,NaN,0,0,924,0,0,21,2006-09-22 06:56:00,2006-09-22 11:50:00,2006-09-22 14:50:00,2006-09-22 20:05:00
3309246,2006,6,24,6,1136.0,1845,1339.0,2036,NW,181,N370NW,243.0,231.0,209.0,1023.0,1011.0,MEM,LAX,1619,11,23,0,NaN,0,1011,0,12,0,0,2006-06-24 11:36:00,2006-06-24 13:39:00,2006-06-24 18:45:00,2006-06-24 20:36:00
423185,2006,1,3,2,831.0,1840,1240.0,2241,NW,1972,N361NW,189.0,181.0,163.0,839.0,831.0,PHX,MSP,1276,6,20,0,NaN,0,831,0,8,0,0,2006-01-03 08:31:00,2006-01-03 12:40:00,2006-01-03 18:40:00,2006-01-03 22:41:00


In [135]:
flights_clean[flights_clean["DepDelay"] > 720].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,487.0,487.000000,487.000000,487.000000,487.000000,487.000000,480.000000,487.000000,487,487.000000,487,480.000000,487.000000,480.000000,480.000000,487.000000,487,487,487.000000,487.000000,487.000000,487.0,0,487.000000,487.000000,487.000000,487.000000,487.000000,487.000000,487,480,487,487
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,402,NaN,NaN,NaN,NaN,NaN,117,62,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,487,480,485,486
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NW,NaN,N532US,NaN,NaN,NaN,NaN,NaN,LAS,MSP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-05 08:35:00,2006-01-05 13:24:00,2006-12-01 06:00:00,2006-09-15 19:44:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213,NaN,5,NaN,NaN,NaN,NaN,NaN,29,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,2
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-02 07:55:00,2006-01-02 09:08:00,2006-01-02 14:50:00,2006-01-02 17:31:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-31 07:33:00,2006-12-31 12:14:00,2006-12-31 18:40:00,2006-12-31 20:05:00
mean,2006.0,6.671458,16.624230,4.244353,1030.020534,1470.059548,1164.389583,1619.942505,NaN,1412.004107,NaN,167.320833,167.295688,139.800000,948.318750,946.162218,NaN,NaN,1053.956879,9.041068,18.852156,0.0,NaN,0.014374,718.059548,64.283368,45.291581,0.061602,106.991786,NaN,NaN,NaN,NaN
std,0.0,3.400385,8.502025,2.025517,518.446980,480.712621,523.407640,515.353541,NaN,1245.547073,NaN,80.676687,88.213376,81.199988,178.680961,177.484783,NaN,NaN,710.776681,11.396807,12.036241,0.0,NaN,0.119148,394.385257,216.105660,185.436181,1.359430,221.673851,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,6.000000,15.000000,8.000000,5.000000,NaN,1.000000,NaN,34.000000,39.000000,19.000000,691.000000,721.000000,NaN,NaN,56.000000,0.000000,4.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,4.000000,10.000000,3.000000,712.000000,1210.000000,901.750000,1325.000000,NaN,535.000000,NaN,109.000000,109.000000,80.750000,811.000000,807.500000,NaN,NaN,510.000000,5.000000,12.000000,0.0,NaN,0.000000,604.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


Many of the entries have large carrier delays, while others have large values for other types of delays. Therefore, the large departure delays are valid values, and no cleaning is necessary.

I will now assess the `TaxiIn` column.

In [136]:
(flights_clean["TaxiIn"] > 120).value_counts()

False    7138384
True         266
Name: TaxiIn, dtype: int64

In [137]:
flights_clean[flights_clean["TaxiIn"] > 120].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
2528726,2006,5,14,7,1837.0,1445,2247.0,1635,XE,2239,N41104,310.0,170.0,158.0,372.0,232.0,GSO,IAH,986,136,16,0,NaN,0,0,0,372,0,0,2006-05-14 18:37:00,2006-05-14 22:47:00,2006-05-14 14:45:00,2006-05-14 16:35:00
4606225,2006,8,7,1,1155.0,1157,1739.0,1451,AA,578,N598AA,284.0,114.0,99.0,168.0,-2.0,ORD,RDU,647,165,20,0,NaN,0,0,0,168,0,0,2006-08-07 11:55:00,2006-08-07 17:39:00,2006-08-07 11:57:00,2006-08-07 14:51:00
615944,2006,2,3,5,1205.0,1130,1635.0,1230,WN,949,N399WN,270.0,60.0,60.0,245.0,35.0,FLL,TPA,197,195,15,0,NaN,0,0,35,210,0,0,2006-02-03 12:05:00,2006-02-03 16:35:00,2006-02-03 11:30:00,2006-02-03 12:30:00
4897558,2006,9,15,5,2004.0,1945,2230.0,2159,OH,5448,N989CA,146.0,134.0,13.0,31.0,19.0,CVG,BGR,906,123,10,0,NaN,0,19,0,12,0,0,2006-09-15 20:04:00,2006-09-15 22:30:00,2006-09-15 19:45:00,2006-09-15 21:59:00
3737842,2006,7,20,4,1636.0,1537,2028.0,1657,UA,488,N367UA,232.0,80.0,47.0,211.0,59.0,DSM,ORD,299,141,44,0,NaN,0,0,0,195,0,16,2006-07-20 16:36:00,2006-07-20 20:28:00,2006-07-20 15:37:00,2006-07-20 16:57:00
4344622,2006,8,10,4,1125.0,1125,2206.0,1955,UA,24,N508UA,461.0,330.0,303.0,131.0,0.0,LAX,JFK,2475,145,13,0,NaN,0,0,0,131,0,0,2006-08-10 11:25:00,2006-08-10 22:06:00,2006-08-10 11:25:00,2006-08-10 19:55:00
4634918,2006,8,12,6,1644.0,1650,1944.0,1754,AA,1788,N4XNAA,180.0,64.0,51.0,110.0,-6.0,AUS,DFW,190,121,8,0,NaN,0,0,0,110,0,0,2006-08-12 16:44:00,2006-08-12 19:44:00,2006-08-12 16:50:00,2006-08-12 17:54:00
4672783,2006,8,10,4,1716.0,1715,15.0,2155,B6,1068,N231JB,359.0,220.0,280.0,140.0,1.0,AUS,JFK,1522,124,15,0,NaN,0,0,0,140,0,0,2006-08-10 17:16:00,2006-08-11 00:15:00,2006-08-10 17:15:00,2006-08-10 21:55:00
2511924,2006,5,14,7,1740.0,1430,2233.0,1634,XE,2004,N11107,293.0,124.0,94.0,359.0,190.0,MCI,IAH,643,190,9,0,NaN,0,0,0,359,0,0,2006-05-14 17:40:00,2006-05-14 22:33:00,2006-05-14 14:30:00,2006-05-14 16:34:00
594911,2006,2,3,5,1138.0,1140,1749.0,1402,US,709,N433US,371.0,142.0,172.0,227.0,-2.0,PIT,TPA,873,188,11,0,NaN,0,0,0,227,0,0,2006-02-03 11:38:00,2006-02-03 17:49:00,2006-02-03 11:40:00,2006-02-03 14:02:00


In [138]:
flights_clean[flights_clean["TaxiIn"] > 120].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,266.0,266.000000,266.000000,266.000000,266.000000,266.000000,262.000000,266.000000,266,266.000000,266,262.000000,266.000000,262.000000,262.000000,266.000000,266,266,266.000000,266.000000,266.000000,266.0,0,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266,262,266,266
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,256,NaN,NaN,NaN,NaN,NaN,108,41,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,264,257,251,255
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B6,NaN,N510JB,NaN,NaN,NaN,NaN,NaN,DFW,JFK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-08-10 16:43:00,2006-05-14 22:29:00,2006-02-12 13:15:00,2006-10-02 15:45:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,NaN,2,NaN,NaN,NaN,NaN,NaN,13,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,3,4
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-02 14:45:00,2006-01-02 17:50:00,2006-01-02 11:21:00,2006-01-02 12:24:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-26 23:42:00,2006-12-27 03:46:00,2006-12-26 21:05:00,2006-12-26 23:10:00
mean,2006.0,6.984962,12.620301,3.789474,1542.774436,1436.251880,1499.652672,1671.875940,NaN,2407.447368,NaN,317.106870,154.872180,141.717557,243.648855,81.560150,NaN,NaN,947.744361,168.838346,25.759398,0.0,NaN,0.015038,10.763158,24.229323,181.872180,0.567669,22.620301,NaN,NaN,NaN,NaN
std,0.0,3.028132,7.331156,2.112149,428.692063,387.808823,836.361481,439.894622,NaN,1988.900312,NaN,107.793847,75.947988,111.100850,170.585443,142.509126,NaN,NaN,665.371483,88.623521,35.790090,0.0,NaN,0.121932,79.935920,101.070329,122.712083,4.446445,52.259319,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,4.000000,410.000000,1.000000,5.000000,NaN,6.000000,NaN,137.000000,50.000000,0.000000,-24.000000,-17.000000,NaN,NaN,116.000000,121.000000,2.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,5.000000,7.000000,2.000000,1301.250000,1155.500000,546.500000,1448.000000,NaN,527.500000,NaN,240.000000,102.250000,72.000000,141.250000,0.000000,NaN,NaN,463.750000,129.000000,10.000000,0.0,NaN,0.000000,0.000000,0.000000,120.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


The 4 entries with the largest taxi in time values were diverted flights. The rest of the entries have significant values for NAS delays. Therefore, the taxi in time values seem to be valid, and do not need cleaning.

I will now move on to the `TaxiOut` column.

In [139]:
(flights_clean["TaxiOut"] > 240).value_counts()

False    7138427
True         223
Name: TaxiOut, dtype: int64

In [140]:
flights_clean[flights_clean["TaxiOut"] > 240].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
3329536,2006,6,21,3,711.0,720,1328.0,914,NW,1112,N329NB,377.0,114.0,91.0,254.0,-9.0,EWR,DTW,487,20,266,0,NaN,0,0,0,254,0,0,2006-06-21 07:11:00,2006-06-21 13:28:00,2006-06-21 07:20:00,2006-06-21 09:14:00
4053820,2006,7,27,4,1815.0,1810,135.0,2140,B6,173,N503JB,620.0,390.0,143.0,235.0,5.0,JFK,SJC,2569,6,291,0,NaN,0,0,0,230,0,5,2006-07-27 18:15:00,2006-07-28 01:35:00,2006-07-27 18:10:00,2006-07-27 21:40:00
5023537,2006,9,28,4,1751.0,1750,2341.0,2000,US,1535,N121UW,470.0,250.0,210.0,221.0,1.0,PHL,DEN,1557,6,254,0,NaN,0,0,0,221,0,0,2006-09-28 17:51:00,2006-09-28 23:41:00,2006-09-28 17:50:00,2006-09-28 20:00:00
3973819,2006,7,28,5,1238.0,1235,1928.0,1545,AA,370,N426AA,350.0,130.0,101.0,223.0,3.0,ORD,LGA,733,6,243,0,NaN,0,0,0,223,0,0,2006-07-28 12:38:00,2006-07-28 19:28:00,2006-07-28 12:35:00,2006-07-28 15:45:00
4049727,2006,7,18,2,2047.0,2045,417.0,2355,B6,87,N558JB,570.0,310.0,137.0,262.0,2.0,JFK,SLC,1989,8,305,0,NaN,0,0,0,262,0,0,2006-07-18 20:47:00,2006-07-19 04:17:00,2006-07-18 20:45:00,2006-07-18 23:55:00
3992433,2006,7,28,5,1212.0,1207,2009.0,1517,AA,1184,N407AA,417.0,130.0,101.0,292.0,5.0,ORD,EWR,719,6,310,0,NaN,0,0,0,292,0,0,2006-07-28 12:12:00,2006-07-28 20:09:00,2006-07-28 12:07:00,2006-07-28 15:17:00
3969793,2006,7,27,4,1958.0,1900,227.0,2213,AA,185,N339AA,569.0,373.0,319.0,254.0,58.0,JFK,LAX,2475,5,245,0,NaN,0,7,0,196,0,51,2006-07-27 19:58:00,2006-07-28 02:27:00,2006-07-27 19:00:00,2006-07-27 22:13:00
3619766,2006,7,27,4,2237.0,1845,442.0,2054,XE,3068,N16541,365.0,129.0,78.0,468.0,232.0,EWR,CLT,529,21,266,0,NaN,0,0,0,266,0,202,2006-07-27 22:37:00,2006-07-28 04:42:00,2006-07-27 18:45:00,2006-07-27 20:54:00
1044904,2006,2,12,7,2140.0,2030,441.0,2330,B6,89,N571JB,601.0,360.0,144.0,311.0,70.0,JFK,ONT,2430,4,273,0,NaN,0,37,33,241,0,0,2006-02-12 21:40:00,2006-02-13 04:41:00,2006-02-12 20:30:00,2006-02-12 23:30:00
2779363,2006,5,31,3,1556.0,1601,2111.0,1728,AA,1217,N460AA,375.0,147.0,113.0,223.0,-5.0,EWR,ORD,719,19,243,0,NaN,0,0,0,223,0,0,2006-05-31 15:56:00,2006-05-31 21:11:00,2006-05-31 16:01:00,2006-05-31 17:28:00


In [141]:
flights_clean[flights_clean["TaxiOut"] > 240].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,223.0,223.000000,223.000000,223.000000,223.00000,223.000000,218.000000,223.000000,223,223.000000,223,218.000000,223.000000,218.000000,218.000000,223.000000,223,223,223.000000,223.000000,223.000000,223.0,0,223.000000,223.000000,223.000000,223.000000,223.0,223.000000,223,218,223,223
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,216,NaN,NaN,NaN,NaN,NaN,38,66,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,216,215,195,212
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AA,NaN,N510JB,NaN,NaN,NaN,NaN,NaN,JFK,ORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-07-27 17:01:00,2006-07-21 21:04:00,2006-06-01 18:00:00,2006-06-01 20:30:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,NaN,3,NaN,NaN,NaN,NaN,NaN,56,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,4,3
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-02 08:59:00,2006-01-02 15:01:00,2006-01-02 08:04:00,2006-01-02 10:44:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-31 06:38:00,2006-12-31 12:08:00,2006-12-31 06:17:00,2006-12-31 07:32:00
mean,2006.0,7.282511,14.215247,3.659193,1626.58296,1575.448430,1180.146789,1785.582960,NaN,1581.511211,NaN,425.940367,175.515695,131.073394,281.417431,31.941704,NaN,NaN,988.452915,8.385650,274.596413,0.0,NaN,0.022422,4.053812,4.080717,250.721973,0.0,16.260090,NaN,NaN,NaN,NaN
std,0.0,2.292960,10.634736,1.512856,338.58407,329.558312,976.271817,380.301695,NaN,1510.910485,NaN,102.924477,105.619701,85.021082,61.961227,64.667488,NaN,NaN,784.706020,14.087697,37.332550,0.0,NaN,0.148383,14.250866,24.397104,71.553003,0.0,41.730847,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,528.00000,530.000000,1.000000,5.000000,NaN,6.000000,NaN,279.000000,45.000000,13.000000,-2.000000,-600.000000,NaN,NaN,67.000000,0.000000,241.000000,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,6.000000,2.000000,3.000000,1526.00000,1440.000000,145.250000,1636.500000,NaN,444.000000,NaN,354.250000,96.000000,72.250000,237.250000,0.000000,NaN,NaN,413.000000,4.000000,251.000000,0.0,NaN,0.000000,0.000000,0.000000,223.500000,0.0,0.000000,NaN,NaN,NaN,NaN


A few of the entries are flights that were diverted. The rest of the flights have large NAS delay times, and corresponding arrival delays, so the taxi out values seem to be valid. No cleaning is necessary.

I will now look at at the `Distance` column.

In [142]:
(flights_clean["Distance"] > 4500).value_counts()

False    7136587
True        2063
Name: Distance, dtype: int64

In [143]:
flights_clean[flights_clean["Distance"] > 4500].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
5320730,2006,9,20,3,1548.0,1545,1924.0,1921,DL,365,N845MH,576.0,576.0,553.0,3.0,3.0,ATL,HNL,4502,8,15,0,NaN,0,0,0,0,0,0,2006-09-20 15:48:00,2006-09-20 19:24:00,2006-09-20 15:45:00,2006-09-20 19:21:00
4689067,2006,8,26,6,2121.0,2130,1247.0,1300,CO,14,N78060,566.0,570.0,540.0,-13.0,-9.0,HNL,EWR,4962,9,17,0,NaN,0,0,0,0,0,0,2006-08-26 21:21:00,2006-08-26 12:47:00,2006-08-26 21:30:00,2006-08-26 13:00:00
5886236,2006,10,18,3,1331.0,1330,1807.0,1805,CO,15,N76054,636.0,635.0,612.0,2.0,1.0,EWR,HNL,4962,5,19,0,NaN,0,0,0,0,0,0,2006-10-18 13:31:00,2006-10-18 18:07:00,2006-10-18 13:30:00,2006-10-18 18:05:00
2853535,2006,5,25,4,2120.0,2130,1255.0,1300,CO,14,N67058,575.0,570.0,547.0,-5.0,-10.0,HNL,EWR,4962,10,18,0,NaN,0,0,0,0,0,0,2006-05-25 21:20:00,2006-05-25 12:55:00,2006-05-25 21:30:00,2006-05-25 13:00:00
1715594,2006,3,29,3,1900.0,1905,823.0,845,DL,316,N844MH,503.0,520.0,473.0,-22.0,-5.0,HNL,ATL,4502,10,20,0,NaN,0,0,0,0,0,0,2006-03-29 19:00:00,2006-03-30 08:23:00,2006-03-29 19:05:00,2006-03-30 08:45:00
4110525,2006,7,21,5,1744.0,1750,843.0,845,DL,640,N828MH,539.0,535.0,513.0,-2.0,-6.0,HNL,ATL,4502,9,17,0,NaN,0,0,0,0,0,0,2006-07-21 17:44:00,2006-07-22 08:43:00,2006-07-21 17:50:00,2006-07-22 08:45:00
1686211,2006,3,9,4,1546.0,1547,2104.0,2054,DL,365,N845MH,618.0,607.0,575.0,10.0,-1.0,ATL,HNL,4502,7,36,0,NaN,0,0,0,0,0,0,2006-03-09 15:46:00,2006-03-09 21:04:00,2006-03-09 15:47:00,2006-03-09 20:54:00
2898443,2006,5,26,5,1612.0,1615,653.0,659,DL,860,N839MH,521.0,524.0,500.0,-6.0,-3.0,HNL,ATL,4502,6,15,0,NaN,0,0,0,0,0,0,2006-05-26 16:12:00,2006-05-27 06:53:00,2006-05-26 16:15:00,2006-05-27 06:59:00
4745161,2006,8,26,6,1206.0,1205,1539.0,1527,DL,855,N831MH,573.0,562.0,547.0,12.0,1.0,ATL,HNL,4502,5,21,0,NaN,0,0,0,0,0,0,2006-08-26 12:06:00,2006-08-26 15:39:00,2006-08-26 12:05:00,2006-08-26 15:27:00
5278481,2006,9,15,5,2123.0,2130,1253.0,1300,CO,14,N67052,570.0,570.0,533.0,-7.0,-7.0,HNL,EWR,4962,10,27,0,NaN,0,0,0,0,0,0,2006-09-15 21:23:00,2006-09-15 12:53:00,2006-09-15 21:30:00,2006-09-15 13:00:00


In [144]:
flights_clean[flights_clean["Distance"] > 4500].describe(include = "all")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
count,2063.0,2063.000000,2063.000000,2063.000000,2058.000000,2063.000000,2053.000000,2063.000000,2063,2063.000000,2063,2053.000000,2063.000000,2053.000000,2053.000000,2058.000000,2063,2063,2063.000000,2063.000000,2063.000000,2063.000000,5,2063.000000,2063.000000,2063.000000,2063.000000,2063.000000,2063.000000,2058,2053,2063,2063
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,54,NaN,NaN,NaN,NaN,NaN,3,3,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,2054,2052,2059,2056
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DL,NaN,N831MH,NaN,NaN,NaN,NaN,NaN,HNL,HNL,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,2006-04-27 21:23:00,2006-07-29 08:46:00,2006-11-19 22:45:00,2006-11-20 13:10:00
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1333,NaN,85,NaN,NaN,NaN,NaN,NaN,1031,1032,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-01-01 12:02:00,2006-01-01 17:05:00,2006-01-01 12:05:00,2006-01-01 17:02:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-12-31 22:46:00,2007-01-01 08:54:00,2006-12-31 22:45:00,2007-01-01 13:10:00
mean,2006.0,6.384876,15.667475,4.017935,1620.197765,1613.431411,1369.326839,1368.978672,NaN,456.387785,NaN,569.208962,571.737276,541.935704,5.587920,8.050534,NaN,NaN,4664.772661,7.427533,19.764905,0.002424,NaN,0.002424,3.856035,0.514300,3.097916,0.019389,3.070286,NaN,NaN,NaN,NaN
std,0.0,3.392695,8.800623,2.000768,339.899903,344.852495,494.825183,480.036037,NaN,433.934566,NaN,46.607143,44.068149,44.054378,33.574721,30.108221,NaN,NaN,220.008972,3.101650,7.663858,0.049183,NaN,0.049183,18.494242,9.396362,9.415617,0.880664,17.193327,NaN,NaN,NaN,NaN
min,2006.0,1.000000,1.000000,1.000000,15.000000,1015.000000,5.000000,643.000000,NaN,14.000000,NaN,473.000000,503.000000,448.000000,-57.000000,-19.000000,NaN,NaN,4502.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
25%,2006.0,4.000000,8.000000,2.000000,1338.000000,1320.000000,842.000000,845.000000,NaN,15.000000,NaN,529.000000,529.000000,505.000000,-11.000000,-5.000000,NaN,NaN,4502.000000,5.000000,15.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


The ranges of the elapsed times show that all the entries have apropriately corresponding durations, and therefore the distances are valid. This is further confirmed by the fact that half the entries in the `Origin` column andhalf the entries in the `Dest` column have the same value, HNL. A web search reveals that this is Daniel K. Inouye International Airport, located in Hawaii, which is off the mainland of the United States. Therefore, it makes sense that flights to or from this airport show long distances and durations.

This concludes the assessment of the columns. I will now clean the entries in `ActualElapsedTime` as indicated in preceding paragraphs.

In [145]:
ActualElapsedTime_outliers_mask = flights_clean["ActualElapsedTime"] > 900
ActualElapsedTime_outliers_mask.value_counts()

False    7138590
True          60
Name: ActualElapsedTime, dtype: int64

In [146]:
flights_clean.drop(flights_clean[ActualElapsedTime_outliers_mask].index, axis = 0, inplace = True)

In [147]:
flights_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [148]:
flights_clean[["ActualElapsedTime", "CRSElapsedTime", "ArrDelay", "DepDelay", "TaxiIn", \
               "TaxiOut", "Distance"]].describe(percentiles = [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,TaxiIn,TaxiOut,Distance
count,7.000475e+06,7.138590e+06,7.000475e+06,7.016660e+06,7.138590e+06,7.138590e+06,7.138590e+06
mean,1.260566e+02,1.271929e+02,8.640427e+00,1.006358e+01,6.405017e+00,1.574271e+01,7.280638e+02
std,7.119734e+01,7.032629e+01,3.645321e+01,3.325261e+01,4.959762e+00,1.126816e+01,5.746217e+02
min,5.000000e+00,-9.700000e+01,-5.920000e+02,-1.123000e+03,0.000000e+00,0.000000e+00,1.100000e+01
5%,4.900000e+01,5.200000e+01,-2.000000e+01,-9.000000e+00,2.000000e+00,6.000000e+00,1.420000e+02
10%,5.700000e+01,6.000000e+01,-1.600000e+01,-7.000000e+00,3.000000e+00,7.000000e+00,1.970000e+02
25%,7.500000e+01,7.600000e+01,-9.000000e+00,-4.000000e+00,4.000000e+00,1.000000e+01,3.160000e+02
50%,1.070000e+02,1.090000e+02,-1.000000e+00,0.000000e+00,5.000000e+00,1.300000e+01,5.740000e+02
75%,1.560000e+02,1.570000e+02,1.300000e+01,9.000000e+00,8.000000e+00,1.800000e+01,9.560000e+02
90%,2.250000e+02,2.250000e+02,4.100000e+01,3.700000e+01,1.100000e+01,2.600000e+01,1.541000e+03


#### **Issue**: Some of the `TailNum` values are 0 or 000000.

##### **Define**

##### **Code**

In [149]:
flights_clean["TailNum"].value_counts().head(10)

0         72704
000000    10009
N308SW     5391
N475HA     3990
N486HA     3913
N481HA     3902
N477HA     3884
N478HA     3879
N480HA     3853
N485HA     3852
Name: TailNum, dtype: int64

In [150]:
TailNum_zeros_mask = flights_clean["TailNum"].isin(["0", "000000"])
TailNum_zeros_mask.value_counts()

False    7055877
True       82713
Name: TailNum, dtype: int64

In [151]:
flights_clean[TailNum_zeros_mask]["UniqueCarrier"].nunique()

14

In [152]:
flights_clean[TailNum_zeros_mask][["Cancelled", "Diverted"]].value_counts()

Cancelled  Diverted
1          0           82544
0          0             158
           1              11
dtype: int64

Nearly all of the flights where only a placeholder tail number was provided were cancelled or diverted. For any analysis of the planes, a subset of the data where tail numbers are provided will be used. Therefore none of these rows will be cleaned.

All the issues pertaining to the `flights` dataframe have now been adressed. I will now confirm that null values in the new timestamp columns are present only where there were null values in the original integer columns.

In [153]:
pd.testing.assert_series_equal(flights_clean["DepTime"].isnull(), \
                               flights_clean["ActualDepTime"].isnull(), \
                               check_names = False)

In [154]:
pd.testing.assert_series_equal(flights_clean["ArrTime"].isnull(), \
                               flights_clean["ActualArrTime"].isnull(), \
                               check_names = False)

In [155]:
pd.testing.assert_series_equal(flights_clean["CRSDepTime"].isnull(), \
                               flights_clean["SchedDepTime"].isnull(), \
                               check_names = False)

In [156]:
pd.testing.assert_series_equal(flights_clean["CRSArrTime"].isnull(), \
                               flights_clean["SchedArrTime"].isnull(), \
                               check_names = False)

The `assert` statements were successful. I will now drop the columns I do not need from the dataframe, and reorder the remaining columns.

In [157]:
flights_clean.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
0,2006,1,11,3,743.0,745,1024.0,1018,US,343,N657AW,281.0,273.0,223.0,6.0,-2.0,ATL,PHX,1587,45,13,0,NaN,0,0,0,0,0,0,2006-01-11 07:43:00,2006-01-11 10:24:00,2006-01-11 07:45:00,2006-01-11 10:18:00
1,2006,1,11,3,1053.0,1053,1313.0,1318,US,613,N834AW,260.0,265.0,214.0,-5.0,0.0,ATL,PHX,1587,27,19,0,NaN,0,0,0,0,0,0,2006-01-11 10:53:00,2006-01-11 13:13:00,2006-01-11 10:53:00,2006-01-11 13:18:00
2,2006,1,11,3,1915.0,1915,2110.0,2133,US,617,N605AW,235.0,258.0,220.0,-23.0,0.0,ATL,PHX,1587,4,11,0,NaN,0,0,0,0,0,0,2006-01-11 19:15:00,2006-01-11 21:10:00,2006-01-11 19:15:00,2006-01-11 21:33:00
3,2006,1,11,3,1753.0,1755,1925.0,1933,US,300,N312AW,152.0,158.0,126.0,-8.0,-2.0,AUS,PHX,872,16,10,0,NaN,0,0,0,0,0,0,2006-01-11 17:53:00,2006-01-11 19:25:00,2006-01-11 17:55:00,2006-01-11 19:33:00
4,2006,1,11,3,824.0,832,1015.0,1015,US,765,N309AW,171.0,163.0,132.0,0.0,-8.0,AUS,PHX,872,27,12,0,NaN,0,0,0,0,0,0,2006-01-11 08:24:00,2006-01-11 10:15:00,2006-01-11 08:32:00,2006-01-11 10:15:00


In [158]:
col_to_drop = ["Year", "Month", "DayofMonth", "DayOfWeek", \
               "DepTime", "CRSDepTime", "ArrTime", "CRSArrTime"]
flights_clean.drop(columns = col_to_drop, inplace = True)
flights_clean.head()

,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,ActualDepTime,ActualArrTime,SchedDepTime,SchedArrTime
0,US,343,N657AW,281.0,273.0,223.0,6.0,-2.0,ATL,PHX,1587,45,13,0,NaN,0,0,0,0,0,0,2006-01-11 07:43:00,2006-01-11 10:24:00,2006-01-11 07:45:00,2006-01-11 10:18:00
1,US,613,N834AW,260.0,265.0,214.0,-5.0,0.0,ATL,PHX,1587,27,19,0,NaN,0,0,0,0,0,0,2006-01-11 10:53:00,2006-01-11 13:13:00,2006-01-11 10:53:00,2006-01-11 13:18:00
2,US,617,N605AW,235.0,258.0,220.0,-23.0,0.0,ATL,PHX,1587,4,11,0,NaN,0,0,0,0,0,0,2006-01-11 19:15:00,2006-01-11 21:10:00,2006-01-11 19:15:00,2006-01-11 21:33:00
3,US,300,N312AW,152.0,158.0,126.0,-8.0,-2.0,AUS,PHX,872,16,10,0,NaN,0,0,0,0,0,0,2006-01-11 17:53:00,2006-01-11 19:25:00,2006-01-11 17:55:00,2006-01-11 19:33:00
4,US,765,N309AW,171.0,163.0,132.0,0.0,-8.0,AUS,PHX,872,27,12,0,NaN,0,0,0,0,0,0,2006-01-11 08:24:00,2006-01-11 10:15:00,2006-01-11 08:32:00,2006-01-11 10:15:00


In [159]:
old_col_order = flights_clean.columns.tolist()
new_col_order = old_col_order[-4:-3] + old_col_order[-2:-1] + \
                old_col_order[-3:-2] + old_col_order[-1:] + old_col_order[:-4]

In [160]:
flights_clean = flights_clean[new_col_order]
flights_clean

,ActualDepTime,SchedDepTime,ActualArrTime,SchedArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2006-01-11 07:43:00,2006-01-11 07:45:00,2006-01-11 10:24:00,2006-01-11 10:18:00,US,343,N657AW,281.0,273.0,223.0,6.0,-2.0,ATL,PHX,1587,45,13,0,NaN,0,0,0,0,0,0
1,2006-01-11 10:53:00,2006-01-11 10:53:00,2006-01-11 13:13:00,2006-01-11 13:18:00,US,613,N834AW,260.0,265.0,214.0,-5.0,0.0,ATL,PHX,1587,27,19,0,NaN,0,0,0,0,0,0
2,2006-01-11 19:15:00,2006-01-11 19:15:00,2006-01-11 21:10:00,2006-01-11 21:33:00,US,617,N605AW,235.0,258.0,220.0,-23.0,0.0,ATL,PHX,1587,4,11,0,NaN,0,0,0,0,0,0
3,2006-01-11 17:53:00,2006-01-11 17:55:00,2006-01-11 19:25:00,2006-01-11 19:33:00,US,300,N312AW,152.0,158.0,126.0,-8.0,-2.0,AUS,PHX,872,16,10,0,NaN,0,0,0,0,0,0
4,2006-01-11 08:24:00,2006-01-11 08:32:00,2006-01-11 10:15:00,2006-01-11 10:15:00,US,765,N309AW,171.0,163.0,132.0,0.0,-8.0,AUS,PHX,872,27,12,0,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7138585,2006-12-29 12:46:00,2006-12-29 12:49:00,2006-12-29 14:52:00,2006-12-29 14:59:00,DL,1675,N905DE,126.0,130.0,108.0,-7.0,-3.0,ATL,EWR,745,6,12,0,NaN,0,0,0,0,0,0
7138586,2006-12-29 12:25:00,2006-12-29 11:55:00,2006-12-29 20:33:00,2006-12-29 19:31:00,DL,1676,N651DL,308.0,276.0,261.0,62.0,30.0,SEA,ATL,2182,12,35,0,NaN,0,30,0,32,0,0
7138587,2006-12-29 21:18:00,2006-12-29 21:15:00,2006-12-29 22:54:00,2006-12-29 22:41:00,DL,1676,N143DA,96.0,86.0,60.0,13.0,3.0,ATL,MCO,403,9,27,0,NaN,0,0,0,0,0,0
7138588,2006-12-29 21:22:00,2006-12-29 21:27:00,2006-12-29 22:09:00,2006-12-29 22:23:00,DL,1677,N904DA,107.0,116.0,81.0,-14.0,-5.0,SLC,SJC,585,5,21,0,NaN,0,0,0,0,0,0


I will now move on to cleaning the `airports` dataframe.

#### **Issue**: Some of the airports in the `airports` dataframe are not in the USA.

##### **Define**
- Identify rows where airports are in countries other than the USA.
- Drop the identified rows.
- Reset the index of the dataframe.

##### **Code**

In [161]:
airports_clean["country"].value_counts()

USA                               3372
Thailand                             1
Palau                                1
N Mariana Islands                    1
Federated States of Micronesia       1
Name: country, dtype: int64

4 entries are for airports outside the USA. I will drop these 4 rows.

In [162]:
airports_clean.drop(airports_clean[airports_clean["country"] != "USA"].index, axis = "index", inplace = True)

In [163]:
# Reset dataframe index
airports_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [164]:
airports_clean[["country"]].value_counts()

country
USA        3372
dtype: int64

This is satisfactory. I move on to addressing the next issue.

#### **Issue**: There are missing values in these columns in the `airports` dataframe: `city` and `state`.

##### **Define**
- Identify entries where values are missing in each column.
- If no flights in the `flights` dataframe depart or land at these airports, drop entries and reset dataframe index. Otherwise, replace missing values.

##### **Code**

I'll first identify rows where there are null values.

In [165]:
airports_clean[["city", "state"]].isnull().value_counts()

city   state
False  False    3364
True   True        8
dtype: int64

Entries where the city is missing are also the entries where the state is missing. I will inspect these entries since there are only a few of them.

In [166]:
airports_nulls_mask = (airports_clean["city"].isnull()) | (airports_clean["state"].isnull())
airports_nulls_mask.value_counts()

False    3364
True        8
dtype: int64

In [167]:
airports_clean[airports_nulls_mask]

,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,NaN,NaN,USA,33.127231,-117.278727
1715,HHH,Hilton Head,NaN,NaN,USA,32.224384,-80.697629
2251,MIB,Minot AFB,NaN,NaN,USA,48.415769,-101.358039
2312,MQT,Marquette County Airport,NaN,NaN,USA,46.353639,-87.395361
2752,RCA,Ellsworth AFB,NaN,NaN,USA,44.145094,-103.103567
2759,RDR,Grand Forks AFB,NaN,NaN,USA,47.961167,-97.401167
2898,SCE,University Park,NaN,NaN,USA,40.851206,-77.846302
2962,SKA,Fairchild AFB,NaN,NaN,USA,47.615058,-117.655803


I will check whether there are any flights that depart from or arrive to these airports. If not, the rows can be removed. Otherwise, the missing cities and states need to be replaced.

In [168]:
(flights_clean["Origin"].isin(airports_clean[airports_nulls_mask]["iata"])).value_counts()

False    7133747
True        4843
Name: Origin, dtype: int64

In [169]:
(flights_clean["Dest"].isin(airports_clean[airports_nulls_mask]["iata"])).value_counts()

False    7133748
True        4842
Name: Dest, dtype: int64

About 5000 flights were made to these airports, and about 5000 more departed from these airports. Since there are only 8 airports, I will manually conduct a web search for the locations of each of the airports, and add the values to two lists: one list for the cities, and another list for the states.

In [170]:
missing_cities = ["Carlsbad", "Hilton Head Island", "Minot", "Negaunee Township", "Box Elder", "Grand Forks", "Benner Township", "Spokane"]
missing_states = ["CA", "SC", "ND", "MI", "SD", "ND", "PA", "WA"]

In [171]:
# Replacing missing cities
airports_clean.loc[airports_clean[airports_clean["city"].isnull()].index, "city"] = missing_cities

In [172]:
airports_clean[airports_clean["state"].isnull()]

,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,Carlsbad,NaN,USA,33.127231,-117.278727
1715,HHH,Hilton Head,Hilton Head Island,NaN,USA,32.224384,-80.697629
2251,MIB,Minot AFB,Minot,NaN,USA,48.415769,-101.358039
2312,MQT,Marquette County Airport,Negaunee Township,NaN,USA,46.353639,-87.395361
2752,RCA,Ellsworth AFB,Box Elder,NaN,USA,44.145094,-103.103567
2759,RDR,Grand Forks AFB,Grand Forks,NaN,USA,47.961167,-97.401167
2898,SCE,University Park,Benner Township,NaN,USA,40.851206,-77.846302
2962,SKA,Fairchild AFB,Spokane,NaN,USA,47.615058,-117.655803


In [173]:
# Replacing missing states
airports_clean.loc[airports_clean[airports_clean["state"].isnull()].index, "state"] = missing_states

##### **Test**

In [174]:
airports_clean.loc[airports_nulls_mask]

,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,Carlsbad,CA,USA,33.127231,-117.278727
1715,HHH,Hilton Head,Hilton Head Island,SC,USA,32.224384,-80.697629
2251,MIB,Minot AFB,Minot,ND,USA,48.415769,-101.358039
2312,MQT,Marquette County Airport,Negaunee Township,MI,USA,46.353639,-87.395361
2752,RCA,Ellsworth AFB,Box Elder,SD,USA,44.145094,-103.103567
2759,RDR,Grand Forks AFB,Grand Forks,ND,USA,47.961167,-97.401167
2898,SCE,University Park,Benner Township,PA,USA,40.851206,-77.846302
2962,SKA,Fairchild AFB,Spokane,WA,USA,47.615058,-117.655803


In [175]:
airports_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3372 entries, 0 to 3371
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   iata     3372 non-null   object 
 1   airport  3372 non-null   object 
 2   city     3372 non-null   object 
 3   state    3372 non-null   object 
 4   country  3372 non-null   object 
 5   lat      3372 non-null   float64
 6   long     3372 non-null   float64
dtypes: float64(2), object(5)
memory usage: 184.5+ KB


This is satisfactory. I move on to the next issue.

#### **Issue**: Some of the airports are duplicated in the `airports` dataframe.

##### **Define**
- Identify where airports are possibly duplicated.
- Determine whether or not they are actual duplicates.

##### **Code**

The duplicated airports in the dataframe cannot be identified by calling the `duplicated` function directly on the dataframe. This is because where duplicates exist, they have different IATA codes, as all values i the `iata` column are unique. Duplicates also have possibly different airport names, and slightly different values for the latitude and longitude. In the example below, the first entry even had null values for city and state before being addressed in the previous section.

In [176]:
airports_clean[airports_clean["iata"].isin(["CLD", "CRQ"])]

,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,Carlsbad,CA,USA,33.127231,-117.278727
1183,CRQ,McClellan-Palomar,Carlsbad,CA,USA,33.128222,-117.280222


I will check for duplicates by rounding the latitude and longitude to 1 decimal place, and checking for duplicates using this pair of columns. This s not foolproof, as different airports located close enough to each other to round off to the same values will show up as duplicates. Hopefully, there are few enough to inspect the entries visually.

In [177]:
# Using "keep = False" to show all copies of duplicated entries
airports_clean[["lat", "long"]].round(1).duplicated(keep = False).sum()

53

There are up to 26 possilbe duplicates in the dataframe. I will view the entries, grouping them by location (the rounded latitude value) for ease of inspection.

In [178]:
airports_dupe_mask = airports_clean[["lat", "long"]].round(1).duplicated(keep = False)
airports_clean[airports_dupe_mask].groupby(airports["lat"].round(1)).apply(display)

,iata,airport,city,state,country,lat,long
2883,SAW,Sawyer,Gwinn,MI,USA,46.353611,-87.395832


,iata,airport,city,state,country,lat,long
1715,HHH,Hilton Head,Hilton Head Island,SC,USA,32.224384,-80.697629
1790,HXD,Hilton Head,Hilton Head Island,SC,USA,32.224361,-80.697472


,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,Carlsbad,CA,USA,33.127231,-117.278727
1183,CRQ,McClellan-Palomar,Carlsbad,CA,USA,33.128222,-117.280222


,iata,airport,city,state,country,lat,long
148,1F0,Downtown Ardmore,Ardmore,OK,USA,34.146989,-97.122652
149,1F1,Lake Murray State Park,Overbrook,OK,USA,34.075097,-97.106679


,iata,airport,city,state,country,lat,long
1625,GMU,Greenville Downtown,Greenville,SC,USA,34.847944,-82.350000
1666,GYH,Donaldson Center,Greenville,SC,USA,34.758319,-82.376415


...

,iata,airport,city,state,country,lat,long
785,AFE,Kake,Kake,AK,USA,56.960481,-133.908269
1974,KAE,Kake SPB,Kake,AK,USA,56.972996,-133.945615


,iata,airport,city,state,country,lat,long
525,5NK,Naknek,Naknek,AK,USA,58.732881,-157.01992


,iata,airport,city,state,country,lat,long
839,ANC,Ted Stevens Anchorage International,Anchorage,AK,USA,61.174320,-149.996186
2066,LHD,Lake Hood SPB,Anchorage,AK,USA,61.180004,-149.971932


,iata,airport,city,state,country,lat,long
1254,DCK,Dahl Creek,Dahl Creek,AK,USA,66.943338,-156.904674
2467,OBU,Kobuk,Kobuk,AK,USA,66.909171,-156.861057


""


I performed a web search for the airports that are possible duplicates. Many of the above were not the same airport, just located close to each other. Alaska, specifically, had several cities where both an airport and a Seaplane Base (SBP) are located, and share a similar name. However, they are separate. Therefore, the conclusion is that no cleaning is necessary for these rows.

I will now move on to the issue in the `carriers` dataframe.

#### **Issue**: There is a missing value in 1 row in the `carriers` dataframe, in the `Code` column.

##### **Define**
- Identify row with missing value.
- If value can be replaced, retain entry. Otherwise, drop entry.

##### **Code**

In [179]:
# Identify row with missing entry
carriers_clean[carriers_clean["Code"].isnull()]

,Code,Description
873,NaN,North American Airlines


A web search revealed that the code for this airline is NA. Searching the original value in the CSV file shows that it is recorded correctly. However, values that are the string NA are read in as null values in Pandas. I will correct the value now.

In [180]:
carriers_clean.loc[carriers_clean["Code"].isnull(), "Code"] = "NA"

##### **Test**

In [181]:
carriers_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491 entries, 0 to 1490
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Code         1491 non-null   object
 1   Description  1491 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


I will now move on to the issues in the `planes` dataframe.

#### **Issue**: There are missing values in all columns of the `planes` dataframe, except the first one.

##### **Define**

##### **Code**

In [182]:
planes_clean.isnull().value_counts()

tailnum  type   manufacturer  issue_date  model  status  aircraft_type  engine_type  year 
False    False  False         False       False  False   False          False        False    4480
         True   True          True        True   True    True           True         True      549
dtype: int64

Null values for all columns are in the same rows. So I will drop all rows.

In [183]:
planes_clean.dropna(axis = 0, inplace = True)

In [184]:
planes_clean.reset_index(drop = True, inplace = True)

##### **Test**

In [185]:
planes_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tailnum        4480 non-null   object
 1   type           4480 non-null   object
 2   manufacturer   4480 non-null   object
 3   issue_date     4480 non-null   object
 4   model          4480 non-null   object
 5   status         4480 non-null   object
 6   aircraft_type  4480 non-null   object
 7   engine_type    4480 non-null   object
 8   year           4480 non-null   object
dtypes: object(9)
memory usage: 315.1+ KB


#### **Issue**: The `issue_date` and `year` columns in the `planes` dataframe have the *string* data type, which is incorrect.

##### **Define**

##### **Code**

In [186]:
planes_clean.head()

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N10156,Corporation,EMBRAER,02/13/2004,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004
1,N102UW,Corporation,AIRBUS INDUSTRIE,05/26/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
2,N10323,Corporation,BOEING,07/01/1997,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986
3,N103US,Corporation,AIRBUS INDUSTRIE,06/18/1999,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
4,N104UA,Corporation,BOEING,01/26/1998,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998


In [187]:
planes_clean = planes_clean.replace("None", pd.NA)
planes_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tailnum        4480 non-null   object
 1   type           4480 non-null   object
 2   manufacturer   4480 non-null   object
 3   issue_date     4479 non-null   object
 4   model          4480 non-null   object
 5   status         4480 non-null   object
 6   aircraft_type  4480 non-null   object
 7   engine_type    4477 non-null   object
 8   year           4335 non-null   object
dtypes: object(9)
memory usage: 315.1+ KB


In [188]:
planes_clean.dropna(axis = 0, inplace = True)

In [189]:
planes_clean.reset_index(drop = True, inplace = True)

In [190]:
planes_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tailnum        4332 non-null   object
 1   type           4332 non-null   object
 2   manufacturer   4332 non-null   object
 3   issue_date     4332 non-null   object
 4   model          4332 non-null   object
 5   status         4332 non-null   object
 6   aircraft_type  4332 non-null   object
 7   engine_type    4332 non-null   object
 8   year           4332 non-null   object
dtypes: object(9)
memory usage: 304.7+ KB


In [191]:
planes_clean["issue_date"] = pd.to_datetime(planes_clean["issue_date"])

In [192]:
planes_clean["year"] = planes_clean["year"].astype(int)

##### **Test**

In [193]:
planes_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332 entries, 0 to 4331
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   tailnum        4332 non-null   object        
 1   type           4332 non-null   object        
 2   manufacturer   4332 non-null   object        
 3   issue_date     4332 non-null   datetime64[ns]
 4   model          4332 non-null   object        
 5   status         4332 non-null   object        
 6   aircraft_type  4332 non-null   object        
 7   engine_type    4332 non-null   object        
 8   year           4332 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 304.7+ KB


In [194]:
planes_clean.head()

,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,N10156,Corporation,EMBRAER,2004-02-13,EMB-145XR,Valid,Fixed Wing Multi-Engine,Turbo-Fan,2004
1,N102UW,Corporation,AIRBUS INDUSTRIE,1999-05-26,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
2,N10323,Corporation,BOEING,1997-07-01,737-3TO,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1986
3,N103US,Corporation,AIRBUS INDUSTRIE,1999-06-18,A320-214,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1999
4,N104UA,Corporation,BOEING,1998-01-26,747-422,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998


All issues for all dataframes have been addressed.

### Feature Engineering
I will now compute two new variables that I will use in the `flights` dataframe. I will adopt the Define-Code-Test framework since it is effective, except instead of applying it to an issue, I will be applying it to a feature/variable.

#### **Feature**: Overall Delay

##### **Define**
- Compute column by subtracting actual elapsed time from scheduled elapsed time.

##### **Code**

In [195]:
successful_clean_mask = flights_clean["ActualArrTime"].notnull()
flights_clean["OverallDelay"] = flights_clean[successful_clean_mask]["ActualElapsedTime"] - flights_clean[successful_clean_mask]["CRSElapsedTime"]

##### **Test**

In [196]:
flights_clean[["ActualDepTime", "SchedDepTime", "ActualArrTime", "SchedArrTime", \
               "ActualElapsedTime", "CRSElapsedTime", "OverallDelay"]].sample(10)

,ActualDepTime,SchedDepTime,ActualArrTime,SchedArrTime,ActualElapsedTime,CRSElapsedTime,OverallDelay
4545360,2006-08-26 17:17:00,2006-08-26 17:20:00,2006-08-26 18:28:00,2006-08-26 18:38:00,191.0,198.0,-7.0
2190237,2006-04-29 12:42:00,2006-04-29 12:42:00,2006-04-29 14:39:00,2006-04-29 14:24:00,177.0,162.0,15.0
4018665,2006-07-09 20:18:00,2006-07-09 20:15:00,2006-07-09 22:30:00,2006-07-09 22:30:00,132.0,135.0,-3.0
5495738,2006-10-25 20:15:00,2006-10-25 19:10:00,2006-10-25 21:47:00,2006-10-25 20:37:00,92.0,87.0,5.0
2152793,NaT,2006-04-08 07:05:00,NaT,2006-04-08 09:38:00,NaN,93.0,NaN
43121,2006-01-05 07:05:00,2006-01-05 07:05:00,2006-01-05 08:53:00,2006-01-05 08:55:00,48.0,50.0,-2.0
4930104,2006-09-14 20:04:00,2006-09-14 18:50:00,2006-09-14 22:30:00,2006-09-14 21:15:00,86.0,85.0,1.0
90237,2006-01-21 14:30:00,2006-01-21 14:35:00,2006-01-21 15:48:00,2006-01-21 15:55:00,78.0,80.0,-2.0
5973570,2006-11-11 16:09:00,2006-11-11 16:10:00,2006-11-11 17:31:00,2006-11-11 17:20:00,82.0,70.0,12.0
896298,2006-02-24 19:56:00,2006-02-24 20:00:00,2006-02-24 20:23:00,2006-02-24 20:29:00,27.0,29.0,-2.0


This is satisfactory. I will now create my second variable.

#### **Issue**: The values in the `FlightNum` column do not each refer to a unique route.

##### **Define**
- Show how `FlightNum` is not unique to a route.
- Create new `FlightRoute` variable, which is a concatenation of the origin airport and destination airport.

##### **Code**

I start by exploring the number of unique flight numbers in the dataframe.

In [197]:
flights_clean["FlightNum"].nunique()

7464

I will now look at the most frequently appearing flight numbers.

In [198]:
flights_clean[["FlightNum"]].value_counts().head(5)

FlightNum
357          5093
490          5077
372          5003
64           4914
476          4768
dtype: int64

I will now look at the number of carrier and origin and destination airport combinations that the most common flight number represented in the year 2006.

In [199]:
flights_clean[flights_clean["FlightNum"] == 357][["UniqueCarrier", "Origin", "Dest"]].value_counts()

UniqueCarrier  Origin  Dest
CO             IAH     RNO     365
AS             OAK     SEA     365
WN             ABQ     STL     363
               STL     BWI     361
B6             JFK     BUR     361
F9             DEN     TPA     348
WN             LAS     ABQ     328
               OAK     LAS     327
FL             LGA     ATL     320
AA             LGA     ORD     305
NW             MSP     SFO     264
UA             ORD     DFW     188
DL             SLC     DFW     150
AS             SEA     GEG     119
US             CLT     CHS     105
               RDU     CLT      90
NW             MKE     MSP      89
UA             DEN     SJC      89
DL             DEN     SLC      63
UA             DEN     SEA      55
               RSW     ORD      53
US             PDX     PHX      51
UA             PHL     ORD      44
WN             BWI     MHT      35
               LAX     ABQ      35
               OAK     LAX      35
UA             BWI     ORD      34
               MCI     DEN 

The flight number has been used by many different carriers. And in several instances, it has been used by the same carrier for more than one route.

This second point is sometimes the case even for a flight number that is used by only one carrier, like the below example.

In [200]:
flights_clean[flights_clean["FlightNum"] == 5986].assign(SchedDepTime = flights_clean["SchedDepTime"].dt.time, \
                                                         SchedArrTime = flights_clean["SchedArrTime"].dt.time)[["SchedDepTime", \
                                                         "SchedArrTime", "UniqueCarrier", "Origin", "Dest"]].value_counts()

SchedDepTime  SchedArrTime  UniqueCarrier  Origin  Dest
16:55:00      18:31:00      OO             SAV     ORD     30
08:51:00      10:12:00      OO             LAX     PHX      5
10:40:00      12:07:00      OO             PHX     LAX      5
dtype: int64

This flight number was used for a total of 40 flights by the same carrier, for 3 different flight routes.

I would like to assess flight delays for distinct flight route. Therefore, I am going to create a new variable that will be a unique identifier for each flight route, and the flight route will be directional, meaning departing from airport A to airport B is not the same route as departing from airport B and landing at airport A. The route will also be independent of carriers, so it will be the same route regardless of which carrier is flying it. This identifier will be a concatenation of the origin and destination airport, with a hyphen between them.

In [201]:
flights_clean["FlightRoute"] = flights_clean["Origin"] + "-" + flights_clean["Dest"]

##### **Test**

In [202]:
flights_clean[["Origin", "Dest", "FlightRoute"]].sample(10)

,Origin,Dest,FlightRoute
5121575,ORD,MKE,ORD-MKE
6891615,KOA,HNL,KOA-HNL
4189399,MDW,PHX,MDW-PHX
5798291,DFW,MCO,DFW-MCO
5748413,HNL,SFO,HNL-SFO
5691166,TPA,ATL,TPA-ATL
2555991,PHL,SFO,PHL-SFO
3540092,MDW,PHL,MDW-PHL
3723843,HNL,SFO,HNL-SFO
4198700,ONT,SJC,ONT-SJC


In [203]:
flights_clean["FlightRoute"].nunique()

4896

### Data Storage

In [205]:
flights_clean.to_csv("flights_master.csv", index = False)
airports_clean.to_csv("airports_master.csv", index = False)
carriers_clean.to_csv("carriers_master.csv", index = False)
planes_clean.to_csv("planes_master.csv", index = False)

### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.


> **Rubric Tip**: The project (Parts I alone) should have at least 15 visualizations distributed over univariate, bivariate, and multivariate plots to explore many relationships in the data set.  Use reasoning to justify the flow of the exploration.



>**Rubric Tip**: Use the "Question-Visualization-Observations" framework  throughout the exploration. This framework involves **asking a question from the data, creating a visualization to find answers, and then recording observations after each visualisation.** 




>**Rubric Tip**: Visualizations should depict the data appropriately so that the plots are easily interpretable. You should choose an appropriate plot type, data encodings, and formatting as needed. The formatting may include setting/adding the title, labels, legend, and comments. Also, do not overplot or incorrectly plot ordinal data.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

## Conclusions
>You can write a summary of the main findings and reflect on the steps taken during the data exploration.



> Remove all Tips mentioned above, before you convert this notebook to PDF/HTML


> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML or PDF` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

